In [1]:
# From: ./.ipynb_checkpoints/training_system-checkpoint.py
from easydict import EasyDict

# from model.cocoop.custom_clip import CustomCLIP
# from utils.datasets import  get_data, base_novel_categories, split_data, CLASS_NAMES
# from utils.debugging import test_step, training_step, eval_step
import clip
from torch.utils.tensorboard import SummaryWriter
from torch.optim import SGD
from torch import nn


class CoCoOpSystem:
    def __init__(self,
                 batch_size=16,
                 num_classes=10,
                 device="cuda:0",
                 learning_rate=0.002,
                 weight_decay=0.0005,
                 momentum=0.9,
                 epochs=2,
                 run_name="exp1",
                 n_ctx=4,
                 ctx_init="",
                 class_token_position="end",
                 csc=False,
                 ):
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.device = device
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.epochs = epochs
        self.run_name = run_name
        self.n_ctx = n_ctx
        self.ctx_init = ctx_init
        self.class_token_position = class_token_position
        self.csc = csc

        # Create a logger for the experiment
        self.writer = SummaryWriter(log_dir=f"runs/{run_name}")

        # Get dataloaders

        self.clip_model, preprocess = clip.load("RN50")
        self.train_set, self.val_set, self.test_set = get_data(transform=preprocess)

        # split classes into base and novel
        self.base_classes, self.novel_classes = base_novel_categories(self.train_set)

        # split the three datasets
        self.train_base, _ = split_data(self.train_set, self.base_classes)
        self.val_base, _ = split_data(self.val_set, self.base_classes)
        self.test_base, self.test_novel = split_data(self.test_set, self.base_classes)

        #self.classnames, _ = embed_dataset_classnames(dataset_name, preprocess=preprocess, model=clip_model)

        resolution = self.clip_model.visual.input_resolution

        cfg = EasyDict()
        # Training configuration
        cfg.TRAINER = EasyDict()
        cfg.TRAINER.COCOOP = EasyDict()
        cfg.TRAINER.COCOOP.N_CTX = self.n_ctx  # Number of context tokens
        cfg.TRAINER.COCOOP.CTX_INIT = self.ctx_init  # Leave empty for random initialization
        cfg.TRAINER.COCOOP.PREC = "fp16"  # Precision for meta network
        cfg.INPUT = EasyDict()
        cfg.INPUT.SIZE = [resolution, resolution]  # Must match CLIP model's input resolution

        # Instantiate the network and move it to the chosen device (GPU)
        self.model = CustomCLIP(
            classnames=[CLASS_NAMES[idx] for idx in self.base_classes],
            cfg=cfg,
            clip_model=self.clip_model,
        ).to(device)

        print("Turning off gradients in both the image and the text encoder")
        for name, param in self.model.named_parameters():
            if "prompt_learner" not in name:
                param.requires_grad_(False)

        print(f"Total parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"Total trainable parameters: {sum(p.numel() for p in self.model.parameters() if p.requires_grad):,}")

        self.optimizer = self.get_optimizer(self.learning_rate, self.weight_decay, self.momentum)
        self.cost_function = nn.CrossEntropyLoss()

    def train(self):
        print("Before training:")
        self.compute_evaluation(-1, base=True)
        # For each epoch, train the network and then compute evaluation results
        for e in range(self.epochs):
            train_loss, train_accuracy = training_step(
                model=self.model,
                dataset=self.train_base,
                optimizer=self.optimizer,
                batch_size=self.batch_size,
                device=self.device,
            )
            val_loss, val_accuracy = eval_step(
                model=self.model,
                dataset=self.val_base,
                cost_function=self.cost_function,
                device=self.device,
                batch_size=self.batch_size,
            )

            self.log_values(e, train_loss, train_accuracy, "train")
            self.log_values(e, val_loss, val_accuracy, "validation")

        print("After training:")
        self.compute_evaluation(self.epochs)
        self.writer.close()

    def compute_evaluation(self, epoch_idx, base=False):
        base_accuracy = test_step(self.model if not base else self.clip_model, self.test_base, self.base_classes, self.batch_size, self.device, label="test", base=base)
        novel_accuracy = test_step(self.model if not base else self.clip_model, self.test_novel, self.novel_classes, self.batch_size, self.device, label="test", base=base)
        # Log to TensorBoard
        self.log_value(epoch_idx,  base_accuracy, "base_classes")
        self.log_value(epoch_idx,  novel_accuracy, "novel_classes")

        return base_accuracy, novel_accuracy

    def get_optimizer(self, lr, wd, momentum):
        optimizer = SGD([
            {
                "params": self.model.parameters()
            }
        ], lr=lr, weight_decay=wd, momentum=momentum)

        return optimizer

    def log_value(self, step,  accuracy, prefix):
        self.writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

    def log_values(self, step, loss, accuracy, prefix):
        self.writer.add_scalar(f"{prefix}/loss", loss, step)
        self.writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [2]:
# From: ./main.py
import yaml
import argparse

# from training_systems.cocoop import CoCoOpSystem
# from training_systems.coop import CoOpSystem
import torch
import os
from datetime import datetime
import pickle
from collections import Counter


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--device', default=os.getenv("DEVICE", "cuda:0"))
    parser.add_argument('--run_name', default=f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
    parser.add_argument('--using_coop', default=False, type=lambda x: x.lower() in ("1", "true", "yes", "true"))
    parser.add_argument('--config', default="train_config.yaml")
    parser.add_argument('--debug', default=True, type=lambda x: x.lower() in ("1", "true", "yes", "true"))
    return parser.parse_args()


# if __name__ == "__main__":
#     # Parse command-line arguments
#     # Assign parsed arguments to variables
#     # Display which device is being used
#     # Handle MPS backend by setting default tensor type to float32
#     # Indicate whether CoOp or CoCoOp is used for training
#     # Load training configuration from YAML file

#     # Initialize and train using CoOpSystem if specified in arguments
#     # Initialize and train using CoCoOpSystem otherwise
#     args = parse_args()

#     device = args.device
#     run_name = args.run_name
#     debug = args.debug
#     use_coop = args.using_coop

#     print(f"Using device: {device}")

#     if torch.backends.mps.is_available():
#         print("\u26a0\ufe0f Forcing float32 due to MPS limitations")
#         torch.set_default_dtype(torch.float32)

#     print(f"Using {'CoOp' if use_coop else 'CoCoOp'} for training")

#     # Load hyperparameters from YAML
#     with open(args.config, "r") as file:
#         config = yaml.safe_load(file)

#     if use_coop:
#         coop_cfg = config['coop']
#         train_sys = CoOpSystem(
#             device=device,
#             run_name=run_name,
#             **coop_cfg
#         )
#     else:
#         cocoop_cfg = config['cocoop']
#         train_sys = CoCoOpSystem(
#             device=device,
#             run_name=run_name,
#             debug=debug,
#             hparams_file=args.config,
#             **cocoop_cfg
#         )

#     train_sys.train()

In [3]:
# From: ./model/cocoop/custom_clip.py
import os.path as osp
from collections import OrderedDict
import math
from contextlib import contextmanager

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.cuda.amp import GradScaler, autocast
# from model.cocoop.prompt_learner import PromptLearner
# from model.cocoop.mlp_adversary import GradientReversalLayer, AdversarialMLP
from clip import clip
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
from easydict import EasyDict

_tokenizer = _Tokenizer()


# class TextEncoder(nn.Module):
#     """
#     TextEncoder wraps CLIP's internal transformer-based text encoder.
#     It encodes tokenized prompts with learned positional embeddings and projects them to the shared feature space.
#     """
#     """
#     Initialize the TextEncoder by extracting relevant CLIP transformer components.
#     """
#     def __init__(self, clip_model):
#         super().__init__()
#         self.transformer = clip_model.transformer
#         self.positional_embedding = clip_model.positional_embedding
#         self.ln_final = clip_model.ln_final
#         self.text_projection = clip_model.text_projection
#         self.dtype = clip_model.dtype
#         print(f"self.dtype={self.dtype}")

#     """
#     Encode a batch of prompts into text features using the transformer and positional embeddings.
#     Returns the end-of-text (EOT) token representations projected into the feature space.
#     """
#     def forward(self, prompts, tokenized_prompts):
#         x = prompts + self.positional_embedding.type(self.dtype)
#         x = x.permute(1, 0, 2)  # NLD -> LND
#         x = self.transformer(x.to(dtype=torch.float32))
#         x = x.permute(1, 0, 2)  # LND -> NLD
#         x = self.ln_final(x).type(self.dtype)

#         # x.shape = [batch_size, n_ctx, transformer.width]
#         # take features from the eot embedding (eot_token is the highest number in each sequence)
#         x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection

#         return x


class CustomCLIP(nn.Module):
    """
    CustomCLIP is a modified CLIP wrapper supporting CoCoOp prompt learning and adversarial training extensions.
    It enables image-text matching via learned prompts and exposes additional methods for class manipulation.
    """
    """
    Initialize the CustomCLIP model with the given configuration, classnames, and base CLIP model.
    Sets up the image and text encoders and prepares the prompt learner.
    """
    def __init__(self, cfg, classnames, clip_model):
        super().__init__()
        self.prompt_learner = PromptLearner(cfg, classnames, clip_model)
        self.tokenized_prompts = self.prompt_learner.tokenized_prompts
        self.image_encoder = clip_model.visual
        self.text_encoder = TextEncoder(clip_model)
        self.logit_scale = clip_model.logit_scale
        self.dtype = clip_model.dtype
        self.clip_model = clip_model
        self.cfg = cfg


    """
    Updates the prompt learner with a new list of classnames and resets its tokenized prompts.
    """
    def change_classnames(self, new_classnames):
        temp_prompt_learner = PromptLearner(
            cfg=self.cfg,
            classnames=new_classnames,
            clip_model=self.clip_model
        )

        self.tokenized_prompts = temp_prompt_learner.tokenized_prompts
        self.prompt_learner.tokenized_prompts = temp_prompt_learner.tokenized_prompts
        self.prompt_learner.token_prefix = temp_prompt_learner.token_prefix
        self.prompt_learner.token_suffix = temp_prompt_learner.token_suffix
        self.prompt_learner.n_cls = len(new_classnames)



    """
    A context manager that temporarily sets new classnames for inference, restoring the original ones afterward.
    """
    @contextmanager
    def temporary_classnames(self, new_classnames):
        # --- Save original state ---
        original_classnames = self.prompt_learner.n_cls
        original_tokenized_prompts = self.tokenized_prompts
        original_token_prefix = self.prompt_learner.token_prefix
        original_token_suffix = self.prompt_learner.token_suffix

        # --- Apply temporary state ---
        temp_prompt_learner = PromptLearner(
            cfg=self.cfg,
            classnames=new_classnames,
            clip_model=self.clip_model
        )

        self.tokenized_prompts = temp_prompt_learner.tokenized_prompts
        self.prompt_learner.tokenized_prompts = temp_prompt_learner.tokenized_prompts
        self.prompt_learner.token_prefix = temp_prompt_learner.token_prefix
        self.prompt_learner.token_suffix = temp_prompt_learner.token_suffix
        self.prompt_learner.n_cls = len(new_classnames)

        try:
            yield
        finally:
            # --- Restore original state ---
            self.tokenized_prompts = original_tokenized_prompts
            self.prompt_learner.tokenized_prompts = original_tokenized_prompts
            self.prompt_learner.token_prefix = original_token_prefix
            self.prompt_learner.token_suffix = original_token_suffix
            self.prompt_learner.n_cls = original_classnames


    """
    Performs a forward pass through the model, computing similarity logits between image features and prompt-conditioned text features.
    Optionally returns loss, image features, and intermediate representations during training.
    """
    def forward(self, image, label=None, get_image_features=False):
        # tokenized_prompts: [num_classes, context_length] (e.g., [10, 77])
        tokenized_prompts = self.tokenized_prompts

        # logit_scale: scalar (e.g., initialized as a learnable parameter like torch.tensor(1/0.07).log())
        logit_scale = self.logit_scale.exp()

        # image: [B, 3, H, W]
        # image_features: [B, D] where D = transformer width (e.g., 512 for ViT-B/32)
        #print(f"image device: {image.device} | image encoder device: {next(self.image_encoder.parameters()).device}")
        image_features = self.image_encoder(image.type(self.dtype))
        if image_features.isnan().any():
            raise ValueError("NaN detected in image_features.")
        # Normalize image features: each row to unit length
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        # prompts: List of [num_classes, context_length, D] (one per image feature)
        # Each element is generated conditioned on an image feature
        prompts, ctx, bias = self.prompt_learner(image_features) # [B , n_cls, n_ctx, D]
        if prompts.isnan().any():
            raise ValueError("NaN detected in prompts.")
        # prompts: [B, n_cls, n_ctx, D] -> [B * n_cls, n_ctx, D]
        logits = []
        all_text_features = []
        selected_text_features = []    
        # Iterate over batch
        for pts_i, imf_i in zip(prompts, image_features):
            # pts_i: [num_classes, context_length, D]
            # tokenized_prompts: [num_classes, context_length]
            # text_features: [num_classes, D]
            text_features = self.text_encoder(pts_i, tokenized_prompts)
            if text_features.isnan().any():
                raise ValueError("NaN detected in text ft.")
            all_text_features.append(text_features)
            # Normalize text features
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # imf_i: [D], text_features.T: [D, num_classes]
            # l_i: [num_classes], similarity scores between image and all class prompts
            l_i = logit_scale * imf_i @ text_features.t()
            # Append l_i (1D tensor) to logits list
            logits.append(l_i)
            best_idx = l_i.argmax()
            best_text_feat = text_features[best_idx]  # [D]
            selected_text_features.append(best_text_feat)

        all_text_features = torch.stack(all_text_features)  # [B, num_classes, D]
        #avarage over num_classes
        avg_text_features = all_text_features.mean(dim=1)

        # Shape: [B, D]
        selected_text_features = torch.stack(selected_text_features)
        # logits: list of B tensors each of shape [num_classes]
        # stacked into a tensor of shape [B, num_classes]
        logits = torch.stack(logits)
        if logits.isnan().any():
            raise ValueError("NaN detected in logits")

        # If in training mode, compute and return cross-entropy loss
        if self.prompt_learner.training:
            # logits: [B, num_classes], label: [B]
            if get_image_features:
                # If get_image_features is True, return logits and image features
                return logits, F.cross_entropy(logits, label), image_features, ctx, bias, avg_text_features, selected_text_features
            else:
                return logits, F.cross_entropy(logits, label)

        # Otherwise, return logits for evaluation: [B, num_classes]
        return logits

    """
    Utility function to print the data types of all parameters and buffers in each model component for debugging.
    """
    def print_all_dtypes(self):
        print(f"CustomCLIP dtype: {self.dtype}")
        print(f"  logit_scale dtype: {getattr(self.logit_scale, 'dtype', type(self.logit_scale))}")
        print(f"  tokenized_prompts dtype: {getattr(self.tokenized_prompts, 'dtype', type(self.tokenized_prompts))}")
        print(f"  image_encoder: {type(self.image_encoder)}")
        for name, param in self.image_encoder.named_parameters():
            print(f"    image_encoder param {name}: {param.dtype}")
        for name, buf in self.image_encoder.named_buffers():
            print(f"    image_encoder buffer {name}: {buf.dtype}")
        print(f"  text_encoder: {type(self.text_encoder)}")
        for name, param in self.text_encoder.named_parameters():
            print(f"    text_encoder param {name}: {param.dtype}")
        for name, buf in self.text_encoder.named_buffers():
            print(f"    text_encoder buffer {name}: {buf.dtype}")
        print(f"  prompt_learner: {type(self.prompt_learner)}")
        for name, param in self.prompt_learner.named_parameters():
            print(f"    prompt_learner param {name}: {param.dtype}")
        for name, buf in self.prompt_learner.named_buffers():
            print(f"    prompt_learner buffer {name}: {buf.dtype}")
        # Also print dtype for ctx, token_prefix, token_suffix if present
        if hasattr(self.prompt_learner, 'ctx'):
            print(f"    prompt_learner.ctx dtype: {self.prompt_learner.ctx.dtype}")
        if hasattr(self.prompt_learner, 'token_prefix'):
            print(f"    prompt_learner.token_prefix dtype: {self.prompt_learner.token_prefix.dtype}")
        if hasattr(self.prompt_learner, 'token_suffix'):
            print(f"    prompt_learner.token_suffix dtype: {self.prompt_learner.token_suffix.dtype}")
        print(f"  clip_model: {type(self.clip_model)}")
        for name, param in self.clip_model.named_parameters():
            print(f"    clip_model param {name}: {param.dtype}")
        for name, buf in self.clip_model.named_buffers():
            print(f"    clip_model buffer {name}: {buf.dtype}")
    """
    Loads a CustomCLIP instance from a saved checkpoint.
    Sets up model configuration, restores state dict, and reassigns classnames.
    """
    @staticmethod
    def load_from_checkpoint(classnames, checkpoint_path, device, n_ctx, clip_model, ctx_8_coop, ctx_4_coop, ctx_init=""):
        ctx_load = (
            ctx_4_coop
            if n_ctx == 4
            else ctx_8_coop
        )
        resolution = clip_model.visual.input_resolution
        cfg = EasyDict(
            {
                "TRAINER": {
                    "COCOOP": {
                        "CTX_LOAD": ctx_load,
                        "N_CTX": n_ctx,
                        "CTX_INIT": ctx_init,
                        "PREC": "fp16",
                    }
                },
                "INPUT": {"SIZE": [resolution, resolution]},
            }
        )
        state_dict = torch.load(checkpoint_path, map_location=device)
        n_cls = state_dict["prompt_learner.token_prefix"].shape[0]

        clip_model, _ = clip.load("ViT-B/16", device=device)

        model = CustomCLIP(cfg, ["X"] * n_cls, clip_model)
        model.load_state_dict(state_dict)
        model.change_classnames(classnames)
        return model

In [4]:
# From: ./model/cocoop/mlp_adversary.py
import torch
import torch.nn as nn
from torch.autograd import Function


class ResidualBlock(nn.Module):
    """
    A residual block with a linear layer, layer normalization, ReLU activation, dropout, and optional identity or projection shortcut.
    """
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = nn.Linear(dim_in, dim_out)
        self.norm = nn.LayerNorm(dim_out)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.2)
        self.residual = (
            nn.Identity() if dim_in == dim_out else nn.Linear(dim_in, dim_out)
        )

    def forward(self, x):
        """
        Forward pass through the residual block with normalization, activation, dropout, and residual connection.
        """
        out = self.linear(x)
        out = self.norm(out)
        out = self.relu(out)
        out = self.drop(out)
        return out + self.residual(x)


class GradientReversalFunction(Function):
    """
    Implements a gradient reversal layer as a custom autograd function, useful in adversarial training.
    """
    @staticmethod
    def forward(ctx, x, lambda_):
        """
        Forward pass that returns the input as-is and stores the lambda factor for use in the backward pass.
        """
        ctx.lambda_ = lambda_
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        """
        Backward pass that reverses the gradient by multiplying it by -lambda.
        """
        return -ctx.lambda_ * grad_output, None


class GradientReversalLayer(nn.Module):
    """
    A wrapper for the GradientReversalFunction to integrate it into a standard nn.Module.
    """
    def __init__(self, lambda_=1.0):
        super().__init__()
        self.lambda_ = lambda_

    def forward(self, x):
        """
        Applies the gradient reversal function with the stored lambda parameter.
        """
        return GradientReversalFunction.apply(x, self.lambda_)


class AdversarialMLP(nn.Module):
    """
    A multi-layer perceptron with optional bias context support, designed for adversarial learning.
    Uses residual blocks for intermediate layers and configurable final output dimension.
    """
    def __init__(self, input_dim, opt, output_dim=1, use_bias_ctx=False, n_ctx=4):
        """
        Initializes the adversarial MLP with given structure and optional bias context input.
        Builds the network using residual blocks and applies Xavier initialization to weights.
        """
        super().__init__()
        hidden_dims = opt.hidden_structure

        layers = []

        if use_bias_ctx:
            # Add a bias context layer if specified
            layers.append(nn.Linear(512*2, hidden_dims[0]))
            layers.append(nn.ReLU())
            for in_dim, out_dim in zip(hidden_dims[:-1], hidden_dims[1:]):
                layers.append(ResidualBlock(in_dim, out_dim))
        else:
            for in_dim, out_dim in zip(hidden_dims[:-1], hidden_dims[1:]):
                layers.append(ResidualBlock(in_dim, out_dim))

        # Final output layer with configurable output_dim
        layers.append(nn.Linear(hidden_dims[-1], output_dim))
        self.model = nn.Sequential(*layers)

        self.model.apply(self.init_weights.__get__(self, AdversarialMLP))

    def forward(self, x):
        """
        Forward pass through the adversarial MLP model.
        """
        return self.model(x)

    def predict(self, x):
        """
        Performs a forward pass and applies a sigmoid activation to the output.
        Squeezes output if it's a single dimension.
        """
        out = self.forward(x)
        return torch.sigmoid(out).squeeze(-1) if out.shape[-1] == 1 else torch.sigmoid(out)

    def init_weights(self, m):
        """
        Initializes weights of linear layers using Xavier uniform distribution and biases to zero.
        """
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

In [ ]:
# From: ./model/cocoop/prompt_learner.py
from collections import OrderedDict
import os
import torch
import torch.nn as nn
from clip import clip
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer

_tokenizer = _Tokenizer()


class PromptLearner(nn.Module):
    """
    The PromptLearner dynamically generates context-conditioned prompts for each class using a meta-network and learned context vectors.
    This module supports both random and pre-initialized context tokens, and computes prompt embeddings compatible with CLIP.
    """
    def __init__(self, cfg, classnames, clip_model):
        """
        Initialize the PromptLearner with configuration parameters, class names, and a CLIP model.
        Loads or initializes context vectors and builds the meta-network for instance-conditioned prompt adaptation.
        """
        super().__init__()
        n_cls = len(classnames)
        n_ctx = cfg.TRAINER.COCOOP.N_CTX
        ctx_init = cfg.TRAINER.COCOOP.CTX_INIT
        dtype = clip_model.dtype
        ctx_dim = clip_model.ln_final.weight.shape[0]
        vis_dim = clip_model.visual.output_dim
        clip_imsize = clip_model.visual.input_resolution
        cfg_imsize = cfg.INPUT.SIZE[0]
        assert cfg_imsize == clip_imsize, f"cfg_imsize ({cfg_imsize}) must equal to clip_imsize ({clip_imsize})"

        if ctx_init:
            # use given words to initialize context vectors
            ctx_init = ctx_init.replace("_", " ")
            n_ctx = len(ctx_init.split(" "))
            prompt = clip.tokenize(ctx_init)
            with torch.no_grad():
                embedding = clip_model.token_embedding(prompt).type(dtype)
            ctx_vectors = embedding[0, 1 : 1 + n_ctx, :]
            prompt_prefix = ctx_init
        else:
            # random initialization
            ctx_vectors = torch.empty(n_ctx, ctx_dim, dtype=dtype)
            nn.init.normal_(ctx_vectors, std=0.02)
            prompt_prefix = " ".join(["X"] * n_ctx)


        self.ctx = nn.Parameter(ctx_vectors)

        # Optional: Load pre-trained ctx from a file
        if hasattr(cfg.TRAINER.COCOOP, "CTX_LOAD") and cfg.TRAINER.COCOOP.CTX_LOAD:
            ctx_path = cfg.TRAINER.COCOOP.CTX_LOAD
            if os.path.isfile(ctx_path):
                #print(f"🔁 Loading ctx from: {ctx_path}")
                state_dict = torch.load(ctx_path, map_location="cpu")
                if "ctx" in state_dict:
                    with torch.no_grad():
                        self.ctx.copy_(state_dict["ctx"])
                else:
                    raise KeyError(f"'ctx' not found in {ctx_path}")
            else:
                raise FileNotFoundError(f"CTX_LOAD path not found: {ctx_path}")
            #print("PROMPT LEARNER LOADED FROM A COOP PRETRAINED ONE")

        self.meta_net = nn.Sequential(OrderedDict([
            ("linear1", nn.Linear(vis_dim, vis_dim // 16)),
            ("relu", nn.ReLU(inplace=True)),
            ("linear2", nn.Linear(vis_dim // 16, ctx_dim))
        ]))

        if cfg.TRAINER.COCOOP.PREC == "fp16" and not torch.backends.mps.is_available():
            self.meta_net.half()
        else:
            print("⚠️ Using float32 for meta_net due to MPS")

        classnames = [name.replace("_", " ") for name in classnames]
        name_lens = [len(_tokenizer.encode(name)) for name in classnames]
        prompts = [prompt_prefix + " " + name + "." for name in classnames]

        tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts])  # (n_cls, n_tkn)
        with torch.no_grad():
            device = clip_model.token_embedding.weight.device

            # Ensure tokenized_prompts is on the right device BEFORE embedding
            tokenized_prompts = tokenized_prompts.to(device)

            embedding = clip_model.token_embedding(tokenized_prompts)

            # Do not convert to fp16 on MPS (Apple doesn't support it fully)
            if device.type == "mps" and dtype == torch.float16:
                print("⚠️ fp16 not fully supported on MPS; using float32 instead")
                dtype = torch.float32

            embedding = embedding.to(dtype)
        # These token vectors will be saved when in save_model(),
        # but they should be ignored in load_model() as we want to use
        # those computed using the current class names
        self.register_buffer("token_prefix", embedding[:, :1, :])  # SOS
        self.register_buffer("token_suffix", embedding[:, 1 + n_ctx :, :])  # CLS, EOS

        self.n_cls = n_cls
        self.n_ctx = n_ctx
        self.tokenized_prompts = tokenized_prompts  # torch.Tensor
        self.name_lens = name_lens

    def construct_prompts(self, ctx, prefix, suffix, label=None):
        """
        Construct the full tokenized prompt from context tokens, prefix (SOS), and suffix (CLS, EOS).
        Optionally uses label indexing for training-time class selection.
        """
        # dim0 is either batch_size (during training) or n_cls (during testing)
        # ctx: context tokens, with shape of (dim0, n_ctx, ctx_dim)
        # prefix: the sos token, with shape of (n_cls, 1, ctx_dim)
        # suffix: remaining tokens, with shape of (n_cls, *, ctx_dim)

        if label is not None:
            prefix = prefix[label]
            suffix = suffix[label]

        prompts = torch.cat(
            [
                prefix,  # (dim0, 1, dim)
                ctx,     # (dim0, n_ctx, dim)
                suffix,  # (dim0, *, dim)
            ],
            dim=1,
        )

        return prompts

    def forward(self, im_features):
        """
        Generate the context-conditioned prompts for each class based on input image features.
        Applies the meta-network to compute per-instance bias vectors, which shift the context tokens.
        Returns the generated prompts along with the original context and computed bias.
        """
        prefix = self.token_prefix
        suffix = self.token_suffix
        ctx = self.ctx                     # (n_ctx, ctx_dim)
        if im_features.isnan().any():
            raise ValueError("NaN in im_features before meta_net")
        
        #print("im_features stats", im_features.min().item(), im_features.max().item(), im_features.norm(dim=1).mean().item())

        meta_net_dtype = next(self.meta_net.parameters()).dtype
        # print(f"meta_net_dtype: {meta_net_dtype}, im_features dtype: {im_features.dtype}")
        bias = self.meta_net(im_features.to(meta_net_dtype))  # (batch, ctx_dim)
        if bias.isnan().any():
            raise ValueError("NaN detected in bias")
        bias = bias.unsqueeze(1)           # (batch, 1, ctx_dim)
        ctx = ctx.unsqueeze(0)             # (1, n_ctx, ctx_dim)
        ctx_shifted = ctx + bias           # (batch, n_ctx, ctx_dim)
        if ctx_shifted.isnan().any():
            raise ValueError("NaN detected in ctx_shifted")
        # Use instance-conditioned context tokens for all classes
        prompts = []
        for ctx_shifted_i in ctx_shifted:
            ctx_i = ctx_shifted_i.unsqueeze(0).expand(self.n_cls, -1, -1) # (n_cls, n_ctx, ctx_dim)
            pts_i = self.construct_prompts(ctx_i, prefix, suffix)  # (n_cls, n_tkn, ctx_dim)
            if pts_i.isnan().any():
                raise ValueError("NaN detected in pts_i")
            prompts.append(pts_i)
        prompts = torch.stack(prompts)

        return prompts, ctx, bias

In [6]:
# From: ./model/coop/custom_clip.py
import clip
import torch
from torch import nn
from torch.nn import functional as F
from contextlib import contextmanager


class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection
        self.dtype = clip_model.dtype

    def forward(self, prompts, tokenized_prompts):
        x = prompts + self.positional_embedding
        x = x.permute(1, 0, 2)  # [batch_size, n_ctx, transformer.width] -> [n_ctx, batch_size, transformer.width]
        x = self.transformer(x.to(dtype=self.transformer.resblocks[0].mlp.c_fc.weight.dtype))
        x = x.permute(1, 0, 2)  # [n_ctx, batch_size, transformer.width] -> [batch_size, n_ctx, transformer.width]
        x = self.ln_final(x)

        # Take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection

        

        return x

    

class PromptLearnerCoOp(nn.Module):
    def __init__(self, cfg, classnames, clip_model):
        super().__init__()
        n_cls = len(classnames)
        n_ctx = cfg.TRAINER.COOP.N_CTX
        ctx_init = cfg.TRAINER.COOP.CTX_INIT
        dtype = clip_model.dtype
        ctx_dim = clip_model.ln_final.weight.shape[0]
        clip_imsize = clip_model.visual.input_resolution
        cfg_imsize = cfg.INPUT.SIZE[0]
        assert cfg_imsize == clip_imsize, f"cfg_imsize ({cfg_imsize}) must equal clip_imsize ({clip_imsize})"

        if ctx_init:
            ctx_init = ctx_init.replace("_", " ")
            n_ctx = len(ctx_init.split(" "))
            prompt = clip.tokenize(ctx_init)
            with torch.no_grad():
                embedding = clip_model.token_embedding(prompt).type(dtype)
            ctx_vectors = embedding[0, 1 : 1 + n_ctx, :]
            prompt_prefix = ctx_init
        else:
            ctx_vectors = torch.empty(n_ctx, ctx_dim, dtype=dtype)
            nn.init.normal_(ctx_vectors, std=0.02)
            prompt_prefix = " ".join(["X"] * n_ctx)

        #print(f'Initial context: "{prompt_prefix}"')
        #print(f"Number of context words (tokens): {n_ctx}")

        self.ctx = nn.Parameter(ctx_vectors)

        classnames = [name.replace("_", " ") for name in classnames]
        prompts = [prompt_prefix + " " + name + "." for name in classnames]
        tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts])

        with torch.no_grad():
            embedding = clip_model.token_embedding(tokenized_prompts.to(clip_model.token_embedding.weight.device))
            embedding = embedding.type(dtype)

        self.register_buffer("token_prefix", embedding[:, :1, :])
        self.register_buffer("token_suffix", embedding[:, 1 + n_ctx :, :])

        self.tokenized_prompts = tokenized_prompts
        self.n_cls = n_cls
        self.n_ctx = n_ctx

    def construct_prompts(self, ctx, prefix, suffix):
        return torch.cat([prefix, ctx, suffix], dim=1)

    def forward(self):
        ctx = self.ctx.unsqueeze(0).expand(self.n_cls, -1, -1)
        return self.construct_prompts(ctx, self.token_prefix, self.token_suffix)


class CustomCLIPCoOp(nn.Module):
    def __init__(self, cfg, classnames, clip_model):
        super().__init__()
        self.prompt_learner = PromptLearnerCoOp(cfg, classnames, clip_model)
        self.clip_model = clip_model
        self.tokenized_prompts = self.prompt_learner.tokenized_prompts
        self.image_encoder = clip_model.visual
        self.text_encoder = TextEncoder(clip_model)
        self.logit_scale = clip_model.logit_scale
        self.cfg = cfg
        self.dtype = clip_model.dtype

    def forward(self, image, label=None):
        logit_scale = self.logit_scale.exp()

        #print("Raw logit_scale:", self.logit_scale.item())
        #print("Exp logit_scale:", logit_scale)

        image_features = self.image_encoder(image.type(self.dtype))
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        prompts = self.prompt_learner()
        text_features = self.text_encoder(prompts.type(self.dtype), self.tokenized_prompts)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        if torch.isnan(image_features).any():
            print("⚠️ NaNs in image_features!")

        if torch.isnan(text_features).any():
            print("⚠️ NaNs in text_features!")

        if torch.isinf(image_features).any():
            print("⚠️ Infs in image_features!")

        if torch.isinf(text_features).any():
            print("⚠️ Infs in text_features!")

        #print("Image feature norm:", image_features.norm(dim=-1).mean().item())
        #print("Text feature norm:", text_features.norm(dim=-1).mean().item())

        logits = logit_scale * image_features @ text_features.t()

        #print(f"is training: {self.prompt_learner.training}, label: {label}")
        
        if label is not None:
            
            return F.cross_entropy(logits, label), logits

        return logits
    @contextmanager
    def temporary_classnames(self, new_classnames):
        # --- Save original state ---
        original_classnames = self.prompt_learner.n_cls
        original_tokenized_prompts = self.tokenized_prompts
        original_token_prefix = self.prompt_learner.token_prefix
        original_token_suffix = self.prompt_learner.token_suffix

        # --- Apply temporary state ---
        temp_prompt_learner = PromptLearnerCoOp(
            cfg=self.cfg,
            classnames=new_classnames,
            clip_model=self.clip_model
        )

        self.tokenized_prompts = temp_prompt_learner.tokenized_prompts
        self.prompt_learner.tokenized_prompts = temp_prompt_learner.tokenized_prompts
        self.prompt_learner.token_prefix = temp_prompt_learner.token_prefix
        self.prompt_learner.token_suffix = temp_prompt_learner.token_suffix
        self.prompt_learner.n_cls = len(new_classnames)

        try:
            yield
        finally:
            # --- Restore original state ---
            self.tokenized_prompts = original_tokenized_prompts
            self.prompt_learner.tokenized_prompts = original_tokenized_prompts
            self.prompt_learner.token_prefix = original_token_prefix
            self.prompt_learner.token_suffix = original_token_suffix
            self.prompt_learner.n_cls = original_classnames

In [7]:
# From: ./training_systems/cocoop.py
"""
Main module for training the CoCoOp system, supporting both base and adversarial training phases.
Includes configuration, data loading, model preparation, and training logic for zero-shot learning with CLIP.
"""

import os
import math
from copy import deepcopy
from statistics import harmonic_mean
from sympy.simplify.cse_main import preprocess_for_cse
import torch
from easydict import EasyDict
from tqdm import tqdm
from torch.utils.tensorboard.writer import SummaryWriter
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
import clip
import numpy as np
import random
import hashlib

# from model.cocoop.custom_clip import CustomCLIP
# from model.cocoop.mlp_adversary import GradientReversalLayer, AdversarialMLP
# from utils import (
#     conditional_clustering,
#     random_clustering,
#     rotating_cluster_generator_shift,
#     get_data,
#     base_novel_categories,
#     split_data,
#     TensorboardLogger,
#     CLASS_NAMES
# )

# from training_systems.training_methods import (
#     Adversarial,
#     KLCoCoOp,
#     BaseCoCoOp,
#     KLCoCoOpV2,
# )
# from training_systems.evaluation_methods import (
#     ZeroShotTestStep,
#     FineTunedTestStep,
#     EvalStep,
# )
# from training_systems.core import DoubleDatasetTrainingMethod

# --- Add this block for reproducibility ---
def set_global_seed(seed):

    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # For CUDA >= 10.2, for full determinism
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    # For PyTorch >= 1.8
    if hasattr(torch, 'use_deterministic_algorithms'):
        torch.use_deterministic_algorithms(True)
# --- End reproducibility block ---


def checksum(model):
    """
    Generate an MD5 checksum of the model's parameters to track changes across training.

    Args:
        model (torch.nn.Module): The model to hash.

    Returns:
        str: MD5 hash string.
    """
    with torch.no_grad():
        all_params = torch.cat(
            [p.view(-1).cpu() for p in model.parameters() if p.requires_grad]
        )
        return hashlib.md5(all_params.numpy().tobytes()).hexdigest()


class CoCoOpSystem:
    """
    Manages the full training process of the CoCoOp model, including configuration, training, evaluation,
    checkpointing, and logging. Supports both base and adversarial training.
    """

    def __init__(
        self,
        *,
        test_batch_size=16,
        pseudo_base_ratio=0.7,
        seed=42,
        device="cuda",
        run_name="exp1",
        cnn_model="ViT-B/16",
        hparams_file,
        optimizer_configs=None,
        skip_tests=None,
        train_base_checkpoint_path=None,
        debug=False,
        prompt_learner_opt=None,
        kl_loss_opt=None,
        adv_training_opt=None,
        base_training_opt=None,
        clustering_opt=None,
        report=False,
        pat=True,
    ):
        """
        Initialize the CoCoOp system, load data, setup the model, loss functions, optimizers, and logger.

        Args:
            batch_size (int): Batch size for training.
            device (str): Device identifier (e.g., 'cuda' or 'cpu').
            run_name (str): Unique name for the training run.
            cnn_model (str): Backbone CLIP model name.
            optimizer_configs (list): Optimizer settings for base and adversarial training.
            skip_tests (list): Booleans to skip testing after each training stage.
            train_base_checkpoint_path (str): Optional path to a pre-trained base model.
            debug (bool): Enables logging of additional debug information.
            prompt_learner_opt, kl_loss_opt, adv_training_opt, base_training_opt: Configuration dictionaries.
        """

        print(f"run_name: {run_name}, using {cnn_model}, pat: {pat}")
        # --- Set global seed for reproducibility ---
        self.seed = seed if seed is not None else 42
        # set_global_seed(self.seed)
        # --- End reproducibility ---
        assert prompt_learner_opt is not None, "prompt_learner_opt must be provided"
        assert kl_loss_opt is not None, "kl_loss_opt must be provided"
        assert adv_training_opt is not None, "adv_training_opt must be provided"
        assert base_training_opt is not None, "base_training_opt must be provided"
        assert clustering_opt is not None, "clustering_opt must be provided"
        assert (
            optimizer_configs is not None and len(optimizer_configs) == 2
        ), "Two optimizer configs must be provided"

        # --- NEW: Pseudo-base/novel split param ---
        self.pseudo_base_ratio = pseudo_base_ratio
        self.pseudo_split_seed = seed

        self.test_batch_size = test_batch_size
        self.device = device
        self.epochs = base_training_opt["epochs"]
        self.run_name = run_name
        self.n_ctx = prompt_learner_opt["n_ctx"]
        self.ctx_init = prompt_learner_opt["ctx_init"]
        self.class_token_position = prompt_learner_opt["class_token_position"]
        self.csc = prompt_learner_opt["csc"]
        self.lambda_kl = kl_loss_opt["lambda_kl"]
        self.double_datasets_kl = kl_loss_opt.get("double_datasets_kl", False)
        self.rotation_period = kl_loss_opt.get("rotation_period", "relative")
        self.warmup_lambda_kl = kl_loss_opt.get("warmup_lambda_kl", 0)  # Add warmup parameter
        self.lambda_adv = adv_training_opt["lambda_adv"]
        self.gaussian_noise = adv_training_opt.get("gaussian_noise", 0.0)
        self.use_bias_ctx = adv_training_opt.get("use_bias_ctx", False)
        self.adv_training_epochs = adv_training_opt["adv_training_epochs"]
        self.cnn_model = cnn_model
        self.warmup_epoch = base_training_opt["warmup_epoch"]
        self.warmup_cons_lr = base_training_opt["warmup_cons_lr"]
        self.using_kl = kl_loss_opt["using_kl"]
        self.grl_lambda = adv_training_opt["grl_lambda"]
        self.mlp_opt = EasyDict(adv_training_opt["mlp_opt"])
        self.skip_tests = (
            skip_tests if skip_tests is not None else [False, False, False]
        )
        self.train_base_checkpoint_path = train_base_checkpoint_path
        self.debug = debug
        self.max_epoch = self.epochs
        self.optimizer_configs = [EasyDict(conf) for conf in optimizer_configs]
        self.warmup_lambda_adv = adv_training_opt["warmup_lambda_adv"]
        self.base_batch_size = base_training_opt["batch_size"]
        self.adv_batch_size = adv_training_opt["batch_size"]
        self.adv_accumulation_steps = adv_training_opt.get("accumulation_steps", 1)
        self.base_accumulation_steps = base_training_opt.get("accumulation_steps", 1)
        self.prompt_learner_warmup_epochs = adv_training_opt["prompt_learner_warmup_epochs"] if "prompt_learner_warmup_epochs" in adv_training_opt else 0
        self.pat = pat
        print(
            "BATCH SIZES: ",
            self.test_batch_size,
            self.base_batch_size,
            self.adv_batch_size,
        )

        self.ignore_no_improvement = adv_training_opt.get("ignore_no_improvement", False)
        if not report:
            self.log_dir = f"runs/CoCoOp/{self.run_name}"
        elif self.pat:
            self.log_dir = f"runs/report/{self.run_name}"
        else:
            self.log_dir = f"runs/report_no_pat/{self.run_name}"

        self.writer = SummaryWriter(log_dir=self.log_dir)
        self.writer.add_text("Hparams yaml file", hparams_file)
        self.logger = TensorboardLogger(self.writer)

        self.logger.log_hparams(
            {
                "batch_size_test": self.test_batch_size,
                "base_batch_size": self.base_batch_size,
                "adv_batch_size": self.adv_batch_size,
                "epochs": self.epochs,
                "n_ctx": self.n_ctx,
                "ctx_init": self.ctx_init,
                "class_token_position": self.class_token_position,
                "csc": self.csc,
                "lambda_kl_first": self.lambda_kl[0],
                "lambda_kl_second": self.lambda_kl[1],
                "warmup_epoch": self.warmup_epoch,
                "warmup_cons_lr": self.warmup_cons_lr,
                "lambda_adv": self.lambda_adv,
                "cnn_model": self.cnn_model,
                "grl_lambda": self.grl_lambda,
                "prompt_learner_warmup_epochs" : self.prompt_learner_warmup_epochs,
                "double_datasets_kl": self.double_datasets_kl,
                "pseudo_base_ratio": self.pseudo_base_ratio,
                "pseudo_split_seed": self.seed,
                "rotation_period": self.rotation_period,
                "warmup_lambda_kl": self.warmup_lambda_kl,
            }
        )

        
        print(f"patience {'disabled' if not self.pat else 'enabled'}")
        # Load model
        self.clip_model, preprocess= clip.load(self.cnn_model)
        self.clip_model = self.clip_model.to(self.device)
        resolution = self.clip_model.visual.input_resolution
        self.train_set, self.val_set, self.test_set = get_data(resolution=resolution, eval_transform=preprocess)
        self.base_classes, self.novel_classes = base_novel_categories(self.train_set)
        # --- NEW: Pseudo-base/novel split ---

        # Helper to split a dataset by class list
        # (moved to method below)

        # Split train_base/val_base into pseudo_base/pseudo_novel
        self.train_base, _ = split_data(self.train_set, self.base_classes)
        self.val_base, self.val_novel = split_data(self.val_set, self.base_classes)
        self.test_base, self.test_novel = split_data(self.test_set, self.base_classes)
        print(f"Base classes length: {len(self.base_classes)}, Novel classes length: {len(self.novel_classes)}")
        print(f"Train base length: {len(self.train_base)}, Val base length: {len(self.val_base)}, Test base length: {len(self.test_base)}")
        print(f"Val novel length: {len(self.val_novel)}, Test novel length: {len(self.test_novel)}")


        self.rotation_steps = int(len(self.base_classes)*(1-self.pseudo_base_ratio))

        self.cluster_generator = rotating_cluster_generator_shift(
            self.base_classes, 
            self.pseudo_base_ratio, 
            steps=self.rotation_steps, 
            seed=self.seed
        )
        
        _, self.pseudo_base_classes, self.pseudo_novel_classes = next(self.cluster_generator)

        self.train_pseudo_base = self.split_by_classes(self.train_base, self.pseudo_base_classes)
        self.train_pseudo_novel = self.split_by_classes(self.train_base, self.pseudo_novel_classes)
        self.val_pseudo_base = self.split_by_classes(self.val_base, self.pseudo_base_classes)
        self.val_pseudo_novel = self.split_by_classes(self.val_base, self.pseudo_novel_classes)

        # --- Model/classnames: only pseudo_base for first phase ---
        ctx_load = (
            "./bin/coop/coop_ctx_4_VIT16.pth"
            if self.n_ctx == 4
            else "./bin/coop/coop_ctx_8_VIT16.pth"
        )
        cfg = EasyDict(
            {
                "TRAINER": {
                    "COCOOP": {
                        "CTX_LOAD": ctx_load,
                        "N_CTX": self.n_ctx,
                        "CTX_INIT": self.ctx_init,
                        "PREC": "fp16",
                    }
                },
                "INPUT": {"SIZE": [resolution, resolution]},
            }
        )
        self.model = CustomCLIP(

            classnames=[CLASS_NAMES[idx] for idx in self.pseudo_base_classes],
            cfg=cfg,
            clip_model=self.clip_model,
        ).to(self.device)

        # Print all dtypes of every component inside CustomCLIP
        # self.model.print_all_dtypes()

        for name, param in self.model.named_parameters():
            if "prompt_learner" not in name:
                param.requires_grad_(False)
            else:
                param.requires_grad_(True)

        self.cost_function = nn.CrossEntropyLoss()
        self.grl = GradientReversalLayer(lambda_=self.grl_lambda)

        clip_dim = self.clip_model.visual.output_dim

        self.mlp_adversary = AdversarialMLP(
            input_dim=clip_dim+len(self.base_classes), opt=self.mlp_opt, output_dim=clustering_opt["n_clusters"], use_bias_ctx=self.use_bias_ctx, n_ctx=self.n_ctx
        ).to(self.device)

        print("mlp adversary struct: ", self.mlp_adversary)
        self.optimizer = self.get_optimizer(self.model, None, self.optimizer_configs[0])
        self.lr_scheduler = LambdaLR(self.optimizer, self._lr_lambda)

        # --- NEW: Cluster dict for adversarial phase ---

        clustering_type = clustering_opt["clustering_type"]

        if clustering_type == "random": 
            # Use random clustering
            self.cls_cluster_dict, _ = random_clustering(
                n_cluster=clustering_opt["n_clusters"],
                seed=self.seed,
                distribution="uniform",
            )
        elif clustering_type == "semantic": 
            # Load clustering information
            self.cls_cluster_dict, _ = conditional_clustering(
                n_cluster=clustering_opt["n_clusters"],
                variance=clustering_opt["variance"],
                cnn=clustering_opt["vision_encoder"],
                device=self.device,
            )
        elif clustering_type == "default":
            # Pseudo_base: cluster 0, pseudo_novel: cluster 1
            self.pseudo_cls_cluster_dict = {c: 0 for c in self.pseudo_base_classes}
            self.pseudo_cls_cluster_dict.update({c: 1 for c in self.pseudo_novel_classes})
            # For adversarial phase, use this dict
            self.cls_cluster_dict = self.pseudo_cls_cluster_dict
        else:
            raise ValueError(f"Unknown clustering type: {clustering_type}")

    def _set_test_methods(self):
        """
        Initializes the zero-shot and fine-tuned test step evaluators for both base and novel class splits.
        """
        self.zero_shot_base_classes_test_method = ZeroShotTestStep(
            model=self.clip_model,
            batch_size=self.test_batch_size,
            categories=self.base_classes,
        )
        self.zero_shot_novel_classes_test_method = ZeroShotTestStep(
            model=self.clip_model,
            batch_size=self.test_batch_size,
            categories=self.novel_classes,
        )
        self.zero_shot_pseudo_base_test_method = ZeroShotTestStep(
            model=self.clip_model,
            batch_size=self.test_batch_size,
            categories=self.pseudo_base_classes,
        )
        self.zero_shot_pseudo_novel_test_method = ZeroShotTestStep(
            model=self.clip_model,
            batch_size=self.test_batch_size,
            categories=self.pseudo_novel_classes,
        )
        self.finetuned_test_method = FineTunedTestStep(
            model=self.model,
            batch_size=self.test_batch_size,
        )

    def _set_eval_method(self):
        """
        Initializes the evaluation method used during validation.
        """
        self.eval_method = EvalStep(
            model=self.model,
            batch_size=self.test_batch_size,
        )

    def _set_train_methods(self):
        """
        Initializes the training method used for both base and adversarial phases, depending on whether KL loss is enabled.
        Chooses between standard and KL-regularized training methods.
        """
        self.adversarial_method = Adversarial(
                lambda_adv=0.05,
                model=self.model,
                optimizer=self.optimizer,
                cls_cluster_dict=self.cls_cluster_dict,
                grl=self.grl,
                mlp_adversary=self.mlp_adversary,
                debug=self.debug,
                tmp_classes=self.base_classes, 
                gaussian_noise=self.gaussian_noise,
                use_bias_ctx=self.use_bias_ctx
            )
            
        if self.using_kl[0]:
            if self.double_datasets_kl:
                self.basic_train_method = KLCoCoOpV2(
                    model=self.model,
                    optimizer=self.optimizer,
                    debug=self.debug,
                    lambda_kl=self.lambda_kl[0],
                )
            else:
                self.basic_train_method = KLCoCoOp(
                    model=self.model,
                    optimizer=self.optimizer,
                    debug=self.debug,
                    lambda_kl=self.lambda_kl[0],
                )
        else:
            self.basic_train_method = BaseCoCoOp(
                model=self.model,
                optimizer=self.optimizer,
                debug=self.debug,
            )

        

    def train(self):
        """
        Execute the full training pipeline: base phase, optionally followed by adversarial training and evaluation.
        """
        self._set_test_methods()
        self._set_eval_method()
        self._set_train_methods()
        if not self.skip_tests[0]:
            print("Doing base accuracy test")
            base_acc, novel_acc = self.compute_evaluation(-1, base=True)
            self._log_final_metrics(
                "Final metrics - CLIP ZERO SHOT",
                base_acc,
                novel_acc,
                -1,
            )

        best_model_path = os.path.join(self.log_dir, "best_model.pth")
        # Ensure all methods are properly initialized prior to the base training phase

        if self.train_base_checkpoint_path is None:
            # Base training phase
            base_end_epoch, _, best_base_epoch = self._train_base_phase(best_model_path)
            self.best_base_epoch = best_base_epoch  # Store for later use if needed
            # Log the best epoch to TensorBoard and logger
            self.writer.add_scalar("Best Base Epoch", best_base_epoch, base_end_epoch)

            # Also print for visibility
            print(f"Best base model found at epoch: {best_base_epoch}")
            if self.epochs != 0:
                if self.pat:
                    print(f"[DEBUG] Loading best model state dict after base phase from: {best_model_path}")
                    self.model.load_state_dict(torch.load(best_model_path))
                    print(f"[DEBUG] Loaded best model with classnames: {self.model.prompt_learner.n_cls} classes")
                else:
                    print(f"[DEBUG] Using last model state from base phase (patience disabled)")
                    print(f"[DEBUG] Model has classnames: {self.model.prompt_learner.n_cls} classes")
                self.save_model(path="./bin/cocoop", prefix="after_first_train_")

            if not self.skip_tests[1]:
                print("Doing base accuracy test")
                base_acc, novel_acc = self.compute_evaluation(base_end_epoch)
                self._log_final_metrics(
                    "Final metrics - After Base Training",
                    base_acc,
                    novel_acc,
                    base_end_epoch,
                )
        else:
            base_end_epoch = 0
            print("Skipping base training")
            print(f"[DEBUG] Loading model state dict from: {self.train_base_checkpoint_path}")
            with self.model.temporary_classnames([CLASS_NAMES[c] for c in self.base_classes]):
                self.model.load_state_dict(torch.load(self.train_base_checkpoint_path))
            print(f"[DEBUG] Loaded model with classnames: {self.model.prompt_learner.n_cls} classes")

        # Re-initialize test/eval/train methods after loading/training model and before adversarial phase
        self._set_eval_method()
        self._set_test_methods()

        self.optimizer = self.get_optimizer(
            self.model, self.mlp_adversary, self.optimizer_configs[1]
        )
        # After changing optimizer, ensure train methods use the new optimizer
        self._set_train_methods()

        checksum1 = None
        if self.debug:
            checksum1 = checksum(self.model)
            # Adversarial phase
            print("Before adv training:", checksum1)

        adv_end_epoch = self._train_adversarial_phase(base_end_epoch, best_model_path)

        if self.debug and checksum1:
            checksum2 = checksum(self.model)
            print("After adv training:", checksum2)
            print(f"checksum1: {checksum1}, checksum2: {checksum2}")
            if checksum1 != checksum2:
                print("Model parameters have changed after adversarial training.")

        if not self.skip_tests[2]:
            print("Doing post-adv. accuracy test")
            base_acc, novel_acc = self.compute_evaluation(adv_end_epoch)
            self._log_final_metrics(
                "Final metrics - After Adversarial Training",
                base_acc,
                novel_acc,
                adv_end_epoch,
            )

        self.logger.close()
        if self.adv_training_epochs != 0:
            self.save_model(path="./bin/cocoop", prefix="after_adv_train_")
            self.save_mlp_adversary()

    def get_next_rotation(self):
        """
        Get the next rotation of the pseudo base and pseudo novel classes.
        """
        _, self.pseudo_base_classes, self.pseudo_novel_classes = next(self.cluster_generator)
        self.train_pseudo_base = self.split_by_classes(self.train_base, self.pseudo_base_classes)
        self.train_pseudo_novel = self.split_by_classes(self.train_base, self.pseudo_novel_classes)
        self.val_pseudo_base = self.split_by_classes(self.val_base, self.pseudo_base_classes)
        self.val_pseudo_novel = self.split_by_classes(self.val_base, self.pseudo_novel_classes)
        return self.pseudo_base_classes, self.pseudo_novel_classes, self.train_pseudo_base, self.train_pseudo_novel, self.val_pseudo_base, self.val_pseudo_novel

    def _train_base_phase(self, best_model_path):
        """
        Train the model using KL regularization only (no adversarial objective).

        Args:
            best_model_path (str): Path to store the best base model.

        Returns:
            Tuple[int, float, int]: Final epoch index, best validation score, and best epoch index.
        """
        best_score = 0.0
        patience = 8
        patience_counter = 0
        c = 0
        method = self.basic_train_method
        evaluation_period = 1 if self.pat else 2
        


        # Initialize rotation tracking
        if self.rotation_period == "random":
            # For random rotation, we'll track when the next rotation should happen
            next_rotation_epoch = random.randint(1, 4)  # Sample from 1 to 4
            rotation_epochs = None  # Not used for random
        else:
            rotation_epochs = int(patience * (3/4)) if self.rotation_period == "relative" else self.rotation_period
            next_rotation_epoch = None  # Not used for fixed/relative
        
        pbar = tqdm(total=self.max_epoch, desc="Base Training")
        best_epoch = -1  # Track the epoch of the best model
        best_epoch_path = best_model_path + ".best_epoch.txt"  # Path to store best epoch

        for e in range(self.max_epoch):

            # Apply lambda_kl warmup if enabled
            if self.warmup_lambda_kl > 0 and (isinstance(method, KLCoCoOp) or isinstance(method, KLCoCoOpV2)):
                progress = (e + 1) / self.warmup_lambda_kl
                current_lambda_kl = 0.1 + (self.lambda_kl[0] - 0.1) * min(progress, 1)
                # Only update lambda_kl for methods that support it (KLCoCoOp and KLCoCoOpV2)
                method.update_lambda_kl(current_lambda_kl)

            if self.using_kl[0]:
                if isinstance(method, DoubleDatasetTrainingMethod):
                    # Check if rotation should happen
                    should_rotate = False
                    if self.rotation_period == "random":
                        if e == next_rotation_epoch:
                            should_rotate = True
                            # Sample next rotation epoch (1 to 4 epochs from now)
                            next_rotation_epoch = e + random.randint(1, 4)
                    else:
                        # Fixed or relative rotation
                        if rotation_epochs is not None and e % rotation_epochs == 0:
                            should_rotate = True
                    
                    if should_rotate:
                        if self.rotation_period == "random":
                            print(f"[DEBUG] Random rotation at epoch {e}, next rotation at epoch {next_rotation_epoch}")
                        (
                            self.pseudo_base_classes, 
                            self.pseudo_novel_classes, 
                            self.train_pseudo_base, 
                            self.train_pseudo_novel, 
                            self.val_pseudo_base, 
                            self.val_pseudo_novel
                        ) = self.get_next_rotation()
                    kl_loss, ce_loss, acc = method.double_datasets_train_step(
                        self.train_pseudo_base,
                        self.train_pseudo_novel,
                        self.base_batch_size,
                        ["pseudo_base", "pseudo_novel KL"],
                        self.pseudo_base_classes,
                        self.pseudo_novel_classes,
                    )
                    total_loss = ce_loss + kl_loss
                else:
                    total_loss, acc, ce_loss, kl_loss = method.train_step(
                        self.train_pseudo_base,
                        self.base_batch_size,
                        classnames=self.pseudo_base_classes,
                        accumulation_steps=self.base_accumulation_steps
                    )
            elif isinstance(method, BaseCoCoOp):
                total_loss, acc = method.train_step(
                    self.train_pseudo_base,
                    self.base_batch_size,
                    classnames=self.pseudo_base_classes,
                    accumulation_steps=self.base_accumulation_steps
                )
                kl_loss = None
                ce_loss = total_loss

            self.logger.log_training_base(
                e,
                self.optimizer.param_groups[0]["lr"],
                ce_loss,
                acc,
                kl_loss,
                total_loss,
            )
            postfix_dict = {
                'lr': self.optimizer.param_groups[0]["lr"],
                'ce_L': ce_loss,
                'kl_L': kl_loss,
                'pat_c': patience_counter,
            }
            if e % evaluation_period == 0:
                base_val_acc, novel_val_acc = self._evaluate_and_log(e)

                score = novel_val_acc

                if (score > best_score) or (not self.pat):
                    best_score = score
                    patience_counter = 0
                    torch.save(self.model.state_dict(), best_model_path)
                    best_epoch = e  # Save the epoch of the best model
                    # Store the best epoch to disk
                    with open(best_epoch_path, "w") as f:
                        f.write(str(best_epoch))
                    # When pat=False, we always save the current model state (last epoch)
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print(f"Early stopping at epoch {e}")
                        break

                postfix_dict["B_val_acc"] = base_val_acc
                postfix_dict["N_val_acc"] = novel_val_acc
                postfix_dict["score"] = score
            
            self.lr_scheduler.step()
            pbar.set_postfix(**postfix_dict)
            pbar.update(1)
            c += 1

        return c, best_score, best_epoch

    def _train_adversarial_phase(self, start_epoch, best_model_path):
        """
        Train the model adversarially with dynamic lambda scheduling and early stopping.

        Args:
            start_epoch (int): Starting epoch index.
            best_model_path (str): Path to save the best adversarial model.

        Returns:
            int: Final epoch index after training.
        """
        best_novel_accuracy = 0.0
        patience = 5
        patience_counter = 0
        at_least_one_improving = False
        warmup_epochs = self.warmup_lambda_adv
        lambda_adv_max = self.lambda_adv
        initial_lambda_adv = 0.05
        pbar = tqdm(total=self.adv_training_epochs, desc="Adversarial Training")

        last_model_state = None  # store last model state

        method = self.adversarial_method

        for e in range(start_epoch, start_epoch + self.adv_training_epochs):
            progress = (e - start_epoch + 1) / warmup_epochs
            new_lambda_adv = initial_lambda_adv + (
                lambda_adv_max - initial_lambda_adv
            ) * min(progress, 1)

            method.update_lambda_adv(new_lambda_adv)

            if (e-start_epoch) < self.prompt_learner_warmup_epochs:
                print(f"[DEBUG] Prompt learner FROZEN at adv epoch {e-start_epoch}")
                for name, param in self.model.named_parameters():
                    if "prompt_learner" in name:
                        param.requires_grad_(False)
            elif (e-start_epoch) == self.prompt_learner_warmup_epochs:
                print(f"prompt learner UNFROZEN at adv epoch {e-start_epoch}")
                for name, param in self.model.named_parameters():
                    if "prompt_learner" in name:
                        param.requires_grad_(True)
                # print the frozen parameters name


            if self.using_kl[1]:
                total_loss, acc, ce_loss, kl_loss, adv_loss = method.train_step(
                    self.train_base,
                    self.base_batch_size,
                    classnames=self.base_classes,
                    accumulation_steps=self.base_accumulation_steps
                )
            else:
                total_loss, acc, ce_loss, adv_loss = method.train_step(
                    self.train_base,
                    self.adv_batch_size,
                    classnames=self.base_classes,
                    accumulation_steps=self.adv_accumulation_steps
                )
                kl_loss = None

            self.logger.log_training_adv(
                e,
                method.lambda_adv,
                ce_loss,
                acc,
                adv_loss,
                ce_loss + adv_loss + (kl_loss if kl_loss else 0.0),
                kl_loss=kl_loss,
            )
            if (e-start_epoch) >= self.prompt_learner_warmup_epochs:

                # Always update last_model_state to track the current state
                last_model_state = deepcopy(self.model.state_dict())

                base_val_acc, novel_val_acc = self._evaluate_and_log(
                    e,
                    is_adv=True,
                )
                if novel_val_acc > best_novel_accuracy or self.ignore_no_improvement or (not self.pat):
                    best_novel_accuracy = novel_val_acc
                    print(f"[DEBUG] Saving model with classnames: {self.model.prompt_learner.n_cls} classes")
                    torch.save(self.model.state_dict(), best_model_path)
                    at_least_one_improving = True
                    patience_counter = 0
                    # When pat=False, we always save the current model state (last epoch)
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print(f"Early stopping adversarial at epoch {e}")
                        break
                pbar.set_postfix(
                    PB_val_acc=base_val_acc,
                    PN_val_acc=novel_val_acc,
                    ce_L=ce_loss,
                    kl_L=kl_loss,
                    adv_L=adv_loss,
                    lr=self.optimizer.param_groups[0]["lr"],
                    pat_c=patience_counter,
                )
            else:

                pbar.set_postfix(
                    adv_loss=adv_loss,
                )
            pbar.update(1)

        # When pat=False, we want to keep the last model state (no loading from checkpoint)
        # When pat=True, we load the best model if there was improvement
        if self.pat and ((at_least_one_improving and self.epochs != 0) or self.ignore_no_improvement):
            print(f"[DEBUG] Loading best model state dict after adversarial phase from: {best_model_path}")
            self.model.load_state_dict(torch.load(best_model_path))
            print(f"[DEBUG] Loaded model with classnames: {self.model.prompt_learner.n_cls} classes")
            print("Loaded best model from adversarial checkpoint (robust, filtered mismatched keys).")
        else:
            print(
                "Using model from last adversarial epoch (patience disabled or no improvement)."
            )
            if last_model_state is not None:
                self.model.load_state_dict(last_model_state)
                print("Loaded last adversarial model state.")
            else:
                # If last_model_state is None (e.g., still in warmup), the model already has the last state
                # But to be safe, we can load from the saved checkpoint which should be the last state when pat=False
                if not self.pat:
                    print(f"[DEBUG] Loading last model state from checkpoint (patience disabled)")
                    self.model.load_state_dict(torch.load(best_model_path))
                    print("Loaded last adversarial model state from checkpoint.")
                else:
                    print("Model already has the last adversarial state (no loading needed).")

        return start_epoch + self.adv_training_epochs

    def _evaluate_and_log(self, epoch, is_adv=False):
        """
        Run validation and log results for both base and novel splits.

        Args:
            epoch (int): Current training epoch.
            is_adv (bool): Whether evaluation is during adversarial training.

        Returns:
            Tuple[float, float]: Accuracy for base and novel classes.
        """

        if is_adv:
            metrics_base = self.eval_method.evaluate(
                dataset=self.val_base,
                classnames=self.base_classes,
                desc_add=" - Base",
            )
            base_val_loss = metrics_base["loss"]
            base_val_acc = metrics_base["accuracy"]

            metrics_novel = self.eval_method.evaluate(
                dataset=self.val_novel,
                classnames=self.novel_classes,
                desc_add=" - Novel",
            )
            novel_val_loss = metrics_novel["loss"]
            novel_val_acc = metrics_novel["accuracy"]   
            
        else:

            metrics_base = self.eval_method.evaluate(
                dataset=self.val_base,
                classnames=self.base_classes,
                desc_add=" - Base",
            )
            base_val_loss = metrics_base["loss"]
            base_val_acc = metrics_base["accuracy"]

            metrics_novel = self.eval_method.evaluate(
                dataset=self.val_novel,
                classnames=self.novel_classes,
                desc_add=" - Novel",
            )
            novel_val_loss = metrics_novel["loss"]
            novel_val_acc = metrics_novel["accuracy"]  

        self.logger.log_validation(
            epoch,
            base_val_loss,
            base_val_acc,
            novel_val_loss,
            novel_val_acc,
            is_adv=is_adv,
        )

        return base_val_acc, novel_val_acc

    def _log_final_metrics(self, tag, base_acc, novel_acc, step):
        """
        Log final test results to TensorBoard.

        Args:
            tag (str): Descriptive tag for the log.
            base_acc (float): Accuracy on base classes.
            novel_acc (float): Accuracy on novel classes.
            step (int): Epoch or step index for this log.
        """
        self.logger.log_final_metrics(tag, base_acc, novel_acc, step)

    def _lr_lambda(self, current_epoch):
        """
        Learning rate scheduler with cosine annealing and warm-up.

        Args:
            current_epoch (int): Epoch index.

        Returns:
            float: Learning rate multiplier.
        """
        if current_epoch < self.warmup_epoch:
            return self.warmup_cons_lr / self.optimizer_configs[0].prompt_lr # type: ignore
        return 0.5 * (
            1
            + math.cos(
                math.pi
                * (current_epoch - self.warmup_epoch)
                / (self.max_epoch - self.warmup_epoch + 1e-7)
            )
        )

    def compute_evaluation(self, epoch_idx, base=False):
        """
        Run evaluation on the test split for both base and novel classes.

        Args:
            epoch_idx (int): Epoch index for logging.
            base (bool): Whether to evaluate the frozen base CLIP model.

        Returns:
            Tuple[float, float]: Base and novel class test accuracy.

        model = self.model if not base else self.clip_model
        base_accuracy = test_step(model, self.test_base, self.base_classes, self.batch_size, self.device, label="test", base=base)
        novel_accuracy = test_step(model, self.test_novel, self.novel_classes, self.batch_size, self.device, label="test", base=base)
        """
        if base:
            base_metrics = self.zero_shot_base_classes_test_method.evaluate(
                dataset=self.test_base,
                desc_add=" - Base Zero Shot",
        
            )
            novel_metrics = self.zero_shot_novel_classes_test_method.evaluate(
                dataset=self.test_novel,
                desc_add=" - Novel Zero Shot",
            )
        else:
            base_metrics = self.finetuned_test_method.evaluate(
                dataset=self.test_base,
                classnames=self.base_classes,
                desc_add=" - Base Fine Tuned",
            )
            novel_metrics = self.finetuned_test_method.evaluate(
                dataset=self.test_novel,
                classnames=self.novel_classes,
                desc_add=" - Novel Fine Tuned",
            )

        base_accuracy = base_metrics["accuracy"]
        novel_accuracy = novel_metrics["accuracy"]
        self.logger.log_test_accuracy(epoch_idx, base_accuracy, "base_classes")
        self.logger.log_test_accuracy(epoch_idx, novel_accuracy, "novel_classes")
        return base_accuracy, novel_accuracy

    def save_model(self, path="./bin/cocoop", prefix=""):
        """
        Save model weights to disk.

        Args:
            path (str): Directory to save the model to.
            prefix (str): Filename prefix to distinguish models.
        """
        os.makedirs(path, exist_ok=True)
        print(f"[DEBUG] Saving model with classnames: {self.model.prompt_learner.n_cls} classes")
        with self.model.temporary_classnames([CLASS_NAMES[idx] for idx in self.base_classes]):
            torch.save(
                self.model.state_dict(), os.path.join(path, f"{prefix}{self.run_name}.pth")
            )

    def save_mlp_adversary(self, path="./bin/cocoop", prefix=""):
        """
        Save the MLP adversary weights to disk.

        Args:
            path (str): Directory to save the MLP adversary model to.
            prefix (str): Filename prefix to distinguish models.
        """
        os.makedirs(path, exist_ok=True)
        torch.save(
            self.mlp_adversary.state_dict(),
            os.path.join(path, f"{prefix}{self.run_name}_mlp_adversary.pth"),
        )

    def get_optimizer(self, model, mlp_adversary, config):
        """
        Build an SGD optimizer with separate learning rates for different parameter groups.

        Args:
            model (torch.nn.Module): Main model.
            mlp_adversary (torch.nn.Module): Optional adversarial MLP.
            config: Optimizer configuration namespace.

        Returns:
            torch.optim.Optimizer: Configured optimizer.
        """
        params = [
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if "prompt_learner" in n and p.requires_grad
                ],
                "lr": config.prompt_lr,
            }
        ]
        if mlp_adversary is not None:
            params.append(
                {
                    "params": mlp_adversary.parameters(),
                    "lr": config.mlp_lr,
                }
            )
        return torch.optim.SGD(
            params, weight_decay=config.weight_decay, momentum=config.momentum
        )

    def split_by_classes(self, dataset, class_list):
        idxs = [i for i, (_, label) in enumerate(dataset) if label in class_list]
        return torch.utils.data.Subset(dataset, idxs)

In [8]:
# From: ./training_systems/coop.py
"""
This module defines the CoOpSystem class for training and evaluating a prompt-tuned CLIP model using the CoOp method.
It includes data loading, training with early stopping, evaluation, model saving/loading, and logging to TensorBoard.
"""

import os

import torch
from easydict import EasyDict
from tqdm import tqdm

# from model.coop.custom_clip import CustomCLIPCoOp
from utils.datasets import get_data, base_novel_categories, split_data, CLASS_NAMES
from utils.training_coop import test_step, training_step, eval_step
import clip
from torch.utils.tensorboard.writer import SummaryWriter
from torch.optim import SGD, Adam
from torch import nn


class CoOpSystem:
    """
    Implements the CoOp prompt tuning system for training and evaluating CLIP-based models.

    Attributes:
        batch_size (int): Number of samples per training batch.
        device (str): Device identifier (e.g., "cuda:0") to run training on.
        learning_rate (float): Learning rate for the optimizer.
        weight_decay (float): Weight decay used in the optimizer.
        momentum (float): Momentum term (if applicable).
        epochs (int): Number of training epochs.
        run_name (str): Identifier for the experiment run (used for logging and file naming).
        n_ctx (int): Number of context tokens for prompt tuning.
        ctx_init (str): Initialization string for context tokens.
        class_token_position (str): Position of the class token in the prompt.
        csc (bool): Whether to use class-specific context.
    """
    def __init__(self,
                 batch_size=16,
                 device="cuda:0",
                 learning_rate=0.002,
                 weight_decay=0.0005,
                 momentum=0.9,
                 epochs=2,
                 run_name="exp1",
                 n_ctx=4,
                 ctx_init="",
                 class_token_position="end",
                 csc=False,
                 ):
        self.batch_size = batch_size
        self.device = device
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.epochs = epochs
        self.run_name = run_name
        self.n_ctx = n_ctx
        self.ctx_init = ctx_init
        self.class_token_position = class_token_position
        self.csc = csc

        # Create a logger for the experiment
        self.writer = SummaryWriter(log_dir=f"runs/CoOp/{run_name}")
        self.writer.add_scalar(f"lr", self.learning_rate, 0)
        self.writer.add_scalar(f"momentum", self.momentum, 0)

        # Get dataloaders

        self.clip_model, _ = clip.load("ViT-B/16", device=self.device)
        self.clip_model = self.clip_model.to(self.device)
        self.clip_model = self.clip_model.float()
        resolution = self.clip_model.visual.input_resolution
        self.train_set, self.val_set, self.test_set = get_data(resolution=resolution)

        # split classes into base and novel
        self.base_classes, self.novel_classes = base_novel_categories(self.train_set)

        # split the three datasets
        self.train_base, _ = split_data(self.train_set, self.base_classes)
        self.val_base, self.val_novel = split_data(self.val_set, self.base_classes)
        self.test_base, self.test_novel = split_data(self.test_set, self.base_classes)

        #self.classnames, _ = embed_dataset_classnames(dataset_name, preprocess=preprocess, model=clip_model)

        resolution = self.clip_model.visual.input_resolution

        cfg = EasyDict()
        # Training configuration
        cfg.TRAINER = EasyDict()
        cfg.TRAINER.COOP = EasyDict()
        cfg.TRAINER.COOP.N_CTX = self.n_ctx  # Number of context tokens
        cfg.TRAINER.COOP.CTX_INIT = self.ctx_init  # Leave empty for random initialization
        cfg.INPUT = EasyDict()
        cfg.INPUT.SIZE = [resolution, resolution]  # Must match CLIP model's input resolution

        # Instantiate the network and move it to the chosen device (GPU)
        self.model = CustomCLIPCoOp(
            classnames=[CLASS_NAMES[idx] for idx in self.base_classes],
            cfg=cfg,
            clip_model=self.clip_model,
        ).to(device)

        print("Turning off gradients in both the image and the text encoder")
        for name, param in self.model.named_parameters():
            if "prompt_learner" in name:
                param.requires_grad_(True)
            else:
                param.requires_grad_(False)

        print(f"Total parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"Total trainable parameters: {sum(p.numel() for p in self.model.parameters() if p.requires_grad):,}")

        self.optimizer = self.get_optimizer(self.learning_rate, self.weight_decay, self.momentum)
        self.cost_function = nn.CrossEntropyLoss()

    def train(self):
        """
        Trains the CoOp model on the base classes with early stopping and logs performance metrics.
        Saves the best model and computes evaluation at the end of training.
        """
        print("Before training:")
        print("Training the model...")
        print_epoch_interval = 1
        pbar = tqdm(total=self.epochs, desc="OVERALL TRAINING", position=0, leave=True)

        best_val_acc = 0.0
        patience = 3
        counter = 0
        best_model_state = None

        for e in range(self.epochs):
            base_train_loss, base_train_accuracy = training_step(
                model=self.model,
                dataset=self.train_base,
                optimizer=self.optimizer,
                batch_size=self.batch_size,
                classnames=self.base_classes,
                device=self.device,
            )

            if e % print_epoch_interval == 0:
                base_val_loss, base_val_accuracy = eval_step(
                    model=self.model,
                    dataset=self.val_base,
                    cost_function=self.cost_function,
                    new_classnames=self.base_classes,
                    device=self.device,
                    batch_size=self.batch_size,
                )

                self.log_values(e, base_train_loss, base_train_accuracy, "train_base")
                self.log_values(e, base_val_loss, base_val_accuracy, "validation_base")

                pbar.set_postfix(train_acc=base_train_accuracy, val_acc=base_val_accuracy)

                # Early stopping check
                if base_val_accuracy > best_val_acc:
                    best_val_acc = base_val_accuracy
                    counter = 0
                    best_model_state = self.model.state_dict()
                else:
                    counter += 1
                    if counter >= patience:
                        print(f"Early stopping at epoch {e}, best validation accuracy: {best_val_acc:.4f}")
                        break

            pbar.update(1)

        # Restore best model if early stopped
        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)

        print("After training:")
        self.compute_evaluation(self.epochs)
        self.writer.close()

        self.save_model()
        self.save_prompt_learner()

    def save_model(self, path="./bin/coop"):
        """
        Saves the entire model's state dictionary to disk under the specified path.

        Args:
            path (str): Directory path where the model checkpoint will be saved.
        """
        #create folder if not exist
        if not os.path.exists(path):
            os.makedirs(path)
        # Save the model
        torch.save(self.model.state_dict(), os.path.join(path, f"{self.run_name}.pth"))

    def save_prompt_learner(self, path="./bin/coop"):
        """
        Saves only the prompt learner component of the model to disk.

        Args:
            path (str): Directory path where the prompt learner checkpoint will be saved.
        """
        # Create folder if not exist
        if not os.path.exists(path):
            os.makedirs(path)
        # Save only the self.ctx parameter of the prompt learner
        ctx_state = {"ctx": self.model.prompt_learner.ctx.detach().cpu()}
        torch.save(ctx_state, os.path.join(path, f"{self.run_name}_prompt_learner.pth"))

    def load_model(self, path="./bin"):
        """
        Loads a saved model checkpoint from disk and sets the model to evaluation mode.

        Args:
            path (str): Directory path from which the model checkpoint will be loaded.
        """
        # Load the model
        self.model.load_state_dict(torch.load(os.path.join(path, f"{self.run_name}.pth")))
        self.model.eval()
        print(f"Model loaded from {path}")

    def compute_evaluation(self, epoch_idx, base=False):
        """
        Evaluates the model (or zero-shot CLIP if base=True) on the base test set and logs accuracy.

        Args:
            epoch_idx (int): Index of the current epoch for logging.
            base (bool): If True, use zero-shot CLIP model for evaluation instead of the trained model.

        Returns:
            float: Accuracy on the base test set.
        """
        base_accuracy = test_step(
            self.model if not base else self.clip_model, 
            self.test_base, 
            self.batch_size, 
            self.device, 
            self.base_classes,
            label="test", 
            base=base
        )
        # Log to TensorBoard
        self.log_value(epoch_idx,  base_accuracy, "base_classes")

        return base_accuracy

    def get_optimizer(self, lr, wd, momentum):
        """
        Instantiates and returns the optimizer for the model parameters.

        Args:
            lr (float): Learning rate.
            wd (float): Weight decay.
            momentum (float): Momentum term (unused for Adam optimizer).

        Returns:
            torch.optim.Optimizer: Configured Adam optimizer instance.
        """
        optimizer = Adam([
            {
                "params": self.model.parameters()
            }
        ], lr=lr, weight_decay=wd)

        return optimizer

    def log_value(self, step,  accuracy, prefix):
        """
        Logs a single scalar value (accuracy) to TensorBoard.

        Args:
            step (int): Training step or epoch index.
            accuracy (float): Accuracy value to log.
            prefix (str): Tag prefix to categorize the metric in TensorBoard.
        """
        self.writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

    def log_values(self, step, loss, accuracy, prefix):
        """
        Logs both loss and accuracy values to TensorBoard.

        Args:
            step (int): Training step or epoch index.
            loss (float): Loss value to log.
            accuracy (float): Accuracy value to log.
            prefix (str): Tag prefix to categorize the metrics in TensorBoard.
        """
        self.writer.add_scalar(f"{prefix}/loss", loss, step)
        self.writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [9]:
# From: ./training_systems/core/DoubleDatasetTrainingMethod.py
"""
 Abstract base class for training methods. it should have forward_backward method to be fulfilled by its children
"""

from abc import ABC, abstractmethod
from typing import Dict, Any, Callable

from torch.utils.data import DataLoader
from tqdm import tqdm

from utils import AverageMeter, ContiguousLabelDataset

class DoubleDatasetTrainingMethod:
    """
    Abstract base class for training methods. it should have forward_backward method to be fulfilled by its children

    The TrainingMethod class serves as an abstract interface that standardizes the structure of different training strategies.
    It defines key methods that all training strategies must implement, such as:
     - `get_metrics`: to initialize and return the performance metrics.
     - `get_data_loader`: to prepare the DataLoader tailored for the specific training strategy.
     - `forward_backward`: to implement the forward and backward passes during training.
     - `debug_metrics_to_pbar_args`: to convert debug information for progress bar visualization.
     - `training_step_return`: to return summary metrics after a training epoch.
     It also provides common functionality like `start_training`, `optimizer_step`, and `train_step` to be reused across concrete training methods.
    """

    def __init__(self, model: Any, optimizer: Any, title: str, debug) -> None:
        """
        Initialize the training method.

        Args:
            model (Any): The model to train.
            optimizer (Any): The optimizer to use for training.
            title (str): Title for identification (e.g., for progress display).
            debug (bool): Flag to enable debug mode for extra logging.
        """
        self.model = model
        self.optimizer = optimizer
        self.title = title
        self.device = next(self.model.parameters()).device
        self.debug = debug

    @abstractmethod
    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Initialize and return a dictionary of performance metrics.

        Returns:
            Dict[str, AverageMeter]: Dictionary mapping metric names to metric trackers.
        """
        pass

    @abstractmethod
    def get_data_loader1(self, dataset, batch_size) -> DataLoader:
        """
        Create a data loader for the training dataset.

        Args:
            dataset: The training dataset.
            batch_size (int): Number of samples per batch.

        Returns:
            DataLoader: PyTorch data loader instance.
        """
        pass
    @abstractmethod
    def get_data_loader2(self, dataset, batch_size) -> DataLoader:
        """
        Create a data loader for the training dataset.

        Args:
            dataset: The training dataset.
            batch_size (int): Number of samples per batch.

        Returns:
            DataLoader: PyTorch data loader instance.
        """
        pass

    @abstractmethod
    def debug_metrics_to_pbar_args1(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Format debug metrics for display in a progress bar.

        Args:
            debug_metrics (Dict[str, float]): Metrics from the current step.

        Returns:
            Dict[str, float]: Formatted metrics for tqdm display.
        """
        pass

    @abstractmethod
    def debug_metrics_to_pbar_args2(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Format debug metrics for display in a progress bar.
        """
        pass

    @abstractmethod
    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Generate summary metrics after completing a training epoch.

        Args:
            metrics (Dict[str, AverageMeter]): Collected training metrics.

        Returns:
            List[float]: List of averaged metric values for reporting.
        """
        pass

    def optimizer_step(self) -> None:
        """
        Apply the optimizer's step and zero the gradients.
        """
        self.optimizer.step()
        self.optimizer.zero_grad()

    def start_training(self) -> None:
        """
        Set the model in training mode.
        """
        self.model.train()

    @abstractmethod
    def forward_backward1(
            self, sample, batch_idx, metrics: Dict[str, AverageMeter], dataset: ContiguousLabelDataset, classes: list[int]
    ) -> Dict[str, float]:
        """
        Execute forward and backward pass, compute loss, and update metrics.

        Args:
            sample: Current batch sample from data loader.
            batch_idx (int): Index of the current batch.
            metrics (Dict[str, AverageMeter]): Metric trackers.
            dataset (ContiguousLabelDataset): Dataset wrapper for label mapping.

        Returns:
            Dict[str, float]: Dictionary of current debug metric values.
        """
        pass

    @abstractmethod
    def forward_backward2(
            self, sample, batch_idx, metrics: Dict[str, AverageMeter], dataset: ContiguousLabelDataset, classes: list[int]
    ) -> Dict[str, float]:
        """
        Execute forward and backward pass, compute loss, and update metrics.
        """

    def double_datasets_train_step(self, dataset1, dataset2, batch_size, names: list[str], classes1, classes2):
        assert len(names) == 2, "Number of names must be 2"
        metrics = self.get_metrics()
        self.start_training()
        tmp_dataset1 = ContiguousLabelDataset(dataset1, classes1)
        tmp_dataset2 = ContiguousLabelDataset(dataset2, classes2)

        dataloader1 = self.get_data_loader1(tmp_dataset1, batch_size)
        dataloader2 = self.get_data_loader2(tmp_dataset2, batch_size)

        pbar = tqdm(dataloader1, desc=f"Training-{self.title}/{names[0]}", position=1, leave=False)
        for batch_idx, sample in enumerate(dataloader1):
            debug_metrics = self.forward_backward1(sample, batch_idx, metrics, tmp_dataset1, classes1)
            pbar.set_postfix(
                self.debug_metrics_to_pbar_args1(debug_metrics)
            )
            pbar.update(1)

        
        pbar = tqdm(dataloader2, desc=f"Training-{self.title}/{names[1]}", position=1, leave=False)
        for batch_idx, sample in enumerate(dataloader2):
            debug_metrics = self.forward_backward2(sample, batch_idx, metrics, tmp_dataset2, classes2)
            pbar.set_postfix(
                self.debug_metrics_to_pbar_args2(debug_metrics)
            )
            pbar.update(1)       

        return self.training_step_return(metrics)






In [10]:
# From: ./training_systems/core/EvaluationMethod.py
from abc import ABC, abstractmethod
from typing import Dict, Any
from torch.utils.data import DataLoader
from utils.metrics import AverageMeter


class EvaluationMethod(ABC):
    """
    Abstract base class for evaluation methods. Standardizes evaluation interface for different strategies.
    """

    def __init__(self, model, batch_size: int = 32, device=None):
        """
        Initialize evaluation method.

        Args:
            model: The model to evaluate.
            batch_size (int): Evaluation batch size.
        """
        self.model = model
        self.device = next(self.model.parameters()).device if device is None else device
        self.batch_size = batch_size

    @abstractmethod
    def evaluate(self, dataset, classnames, desc_add="") -> Dict[str, float]:
        """
        Perform the evaluation.

        Args:
            dataset: The dataset to evaluate on.

        Returns:
            Dict[str, float]: Dictionary with evaluation results (e.g., accuracy, loss).
        """
        pass

In [11]:
# From: ./training_systems/core/TrainingMethod.py
"""
 Abstract base class for training methods. it should have forward_backward method to be fulfilled by its children
"""

from abc import ABC, abstractmethod
from typing import Dict, Any, Callable

from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

from utils import AverageMeter, ContiguousLabelDataset


class TrainingMethod(ABC):
    """
    Abstract base class for training methods. it should have forward_backward method to be fulfilled by its children

    The TrainingMethod class serves as an abstract interface that standardizes the structure of different training strategies.
    It defines key methods that all training strategies must implement, such as:
     - `get_metrics`: to initialize and return the performance metrics.
     - `get_data_loader`: to prepare the DataLoader tailored for the specific training strategy.
     - `forward_backward`: to implement the forward and backward passes during training.
     - `debug_metrics_to_pbar_args`: to convert debug information for progress bar visualization.
     - `training_step_return`: to return summary metrics after a training epoch.
     It also provides common functionality like `start_training`, `optimizer_step`, and `train_step` to be reused across concrete training methods.
    """

    def __init__(self, model: Any, optimizer: Any, title: str, debug) -> None:
        """
        Initialize the training method.

        Args:
            model (Any): The model to train.
            optimizer (Any): The optimizer to use for training.
            title (str): Title for identification (e.g., for progress display).
            debug (bool): Flag to enable debug mode for extra logging.
        """
        self.model = model
        self.optimizer = optimizer
        self.title = title
        self.device = next(self.model.parameters()).device
        self.debug = debug
        self.mlp_adversary = None

    @abstractmethod
    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Initialize and return a dictionary of performance metrics.

        Returns:
            Dict[str, AverageMeter]: Dictionary mapping metric names to metric trackers.
        """
        pass

    @abstractmethod
    def get_data_loader(self, dataset, batch_size) -> DataLoader:
        """
        Create a data loader for the training dataset.

        Args:
            dataset: The training dataset.
            batch_size (int): Number of samples per batch.

        Returns:
            DataLoader: PyTorch data loader instance.
        """
        pass

    @abstractmethod
    def forward_backward(
            self, sample, batch_idx, metrics: Dict[str, AverageMeter], dataset: ContiguousLabelDataset, accumulation_steps: int = 1, step: int = 0
    ) -> Dict[str, float]:
        """
        Execute forward and backward pass, compute loss, and update metrics.

        Args:
            sample: Current batch sample from data loader.
            batch_idx (int): Index of the current batch.
            metrics (Dict[str, AverageMeter]): Metric trackers.
            dataset (ContiguousLabelDataset): Dataset wrapper for label mapping.

        Returns:
            Dict[str, float]: Dictionary of current debug metric values.
        """
        pass

    @abstractmethod
    def debug_metrics_to_pbar_args(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Format debug metrics for display in a progress bar.

        Args:
            debug_metrics (Dict[str, float]): Metrics from the current step.

        Returns:
            Dict[str, float]: Formatted metrics for tqdm display.
        """
        pass

    @abstractmethod
    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Generate summary metrics after completing a training epoch.

        Args:
            metrics (Dict[str, AverageMeter]): Collected training metrics.

        Returns:
            List[float]: List of averaged metric values for reporting.
        """

    def optimizer_step(self) -> None:
        """
        Apply the optimizer's step and zero the gradients.
        """
        self.optimizer.step()
        self.optimizer.zero_grad()

    def start_training(self) -> None:
        """
        Set the model in training mode.
        """
        self.model.train()

    def train_step(self, dataset, batch_size, classnames, accumulation_steps: int = 1):
        """
        Perform a complete training epoch, including data loading, training, and metric collection.

        Args:
            dataset: Dataset used for training.
            batch_size (int): Number of samples per training batch.

        Returns:
            List[float]: Averaged values for each tracked metric after the epoch.
        """
        metrics = self.get_metrics()
        self.start_training()
        tmp_dataset = ContiguousLabelDataset(dataset, classnames)
        dataloader = self.get_data_loader(tmp_dataset, batch_size)
        pbar = tqdm(dataloader, desc=f"Training-{self.title}", position=1, leave=False)
        for batch_idx, sample in enumerate(dataloader):
            debug_metrics = self.forward_backward(sample, batch_idx, metrics, tmp_dataset, accumulation_steps=accumulation_steps, step=batch_idx)
            pbar.set_postfix(
                self.debug_metrics_to_pbar_args(debug_metrics)
            )
            pbar.update(1)
        if accumulation_steps > 1:
            if (batch_idx + 1) % accumulation_steps != 0:
                if self.mlp_adversary is not None:
                    torch.nn.utils.clip_grad_norm_(
                        list(self.model.parameters()) + list(self.mlp_adversary.parameters()),
                        max_norm=1.0,
                        norm_type=2.0,
                        error_if_nonfinite=True
                    )

                self.optimizer.step()
                self.optimizer.zero_grad()
        
        return self.training_step_return(metrics)








In [12]:
# From: ./training_systems/evaluation_methods/EvalStep.py

from typing import Dict
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
import torch

# from training_systems.core import EvaluationMethod
# from utils import ContiguousLabelDataset, CLASS_NAMES, AverageMeter


class EvalStep(EvaluationMethod):
    """
    Generic evaluation step for models that support temporary class name modification.
    """
    @torch.no_grad()
    def evaluate(self, dataset, classnames, desc_add="") -> Dict[str, float]:
        """
        Evaluate model performance on the provided dataset.

        Args:
            dataset: Dataset for evaluation.
            new_classnames (list, optional): New class names to apply temporarily.
            desc_add (str): Suffix to append to tqdm description.

        Returns:
            Dict[str, float]: Dictionary containing average loss and accuracy.
        """
        self.model.eval()
        loss_meter = AverageMeter()
        accuracy_meter = AverageMeter()
        tmp_dataset = ContiguousLabelDataset(dataset, classnames)
        dataloader = DataLoader(tmp_dataset, batch_size=self.batch_size, shuffle=False, num_workers=1)

        remapped_classnames = [ CLASS_NAMES[ tmp_dataset.idx2cat[i] ] for i in range(len(tmp_dataset.idx2cat)) ]
        with self.model.temporary_classnames(remapped_classnames):
            self.walk(loss_meter, accuracy_meter, dataloader, desc_add)

        return {"loss": loss_meter.avg, "accuracy": accuracy_meter.avg}

    @torch.no_grad()
    def walk(self, loss_meter, accuracy_meter, dataloader, desc_add=""):
        """
        Perform the evaluation loop over the dataset.

        Args:
            loss_meter: Tracks average loss.
            accuracy_meter: Tracks average accuracy.
            dataloader: DataLoader to iterate over.
            desc_add (str): Additional string to append to tqdm description.
        """
        for images, targets in tqdm(dataloader, desc="Validation" + desc_add, position=1, leave=False):
            images = images.to(self.device)
            targets = targets.to(self.device)
            logits = self.model(images)
            loss = F.cross_entropy(logits, targets)
            predictions = logits.argmax(dim=-1)
            correct = (predictions == targets).sum().item()
            loss_meter.update(loss.item(), n=targets.size(0))
            accuracy_meter.update(correct, n=targets.size(0), raw=True)

In [13]:
# From: ./training_systems/evaluation_methods/TestSteps.py
from typing import Dict

import clip
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
import torch

# from training_systems.core import EvaluationMethod
# from utils import ContiguousLabelDataset, CLASS_NAMES, AverageMeter


class ZeroShotTestStep(EvaluationMethod):
    """
    Evaluation method for models that have been fine-tuned (e.g., CoCoOp or adversarial models).
    """

    def __init__(self, model, batch_size, categories):
        super().__init__(model, batch_size)
        self.categories = categories
        # here we apply the standard CLIP template used for oxford flowers to all categories
        text_inputs = clip.tokenize(
            [f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in self.categories]
        ).to(self.device)
        with torch.no_grad():
            # we can encode the text features once as they are shared for all images
            # therefore we do it outside the evaluation loop
            self.text_features = self.model.encode_text(text_inputs)
            # and here we normalize them (standard pratice with CLIP)
            self.text_features /= self.text_features.norm(dim=-1, keepdim=True)

    @torch.no_grad()
    def evaluate(self, dataset, desc_add="") -> Dict[str, float]:
        self.model.eval()
        accuracy_meter = AverageMeter()
        tmp_dataset = ContiguousLabelDataset(dataset, self.categories)
        dataloader = DataLoader(tmp_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)
        # Remap labels into a contiguous set starting from zero
        self.walk(dataloader, accuracy_meter, desc_add)

        return {"accuracy": accuracy_meter.avg}

    @torch.no_grad()
    def walk(self, dataloader, accuracy_meter, desc_add):
        pbar = tqdm(total=len(dataloader), desc="Test (Zero Shots) " + desc_add, position=1, leave=False)
        for image, target in dataloader:
            # base categories range from 0 to 50, while novel ones from 51 to 101
            # therefore we must map categories to the [0, 50], otherwise we will have wrong predictions
            # Map targets in contiguous set starting from zero
            # Labels needs to be .long() in pytorch
            
            image = image.to(self.device)
            target = target.to(self.device)

            # forward image through CLIP image encoder
            image_features = self.model.encode_image(image)
            # and normalize
            image_features /= image_features.norm(dim=-1, keepdim=True)

            # here cosine similarity between image and text features and keep the argmax for every row (every image)
            predicted_class = (image_features @ self.text_features.T).argmax(dim=-1)
            # now we check which are correct, and sum them (False == 0, True == 1)

            correct = (predicted_class == target).sum().item()
            accuracy_meter.update(correct, n=target.size(0), raw=True)
            pbar.set_postfix({
                "accuracy" : accuracy_meter.avg
            })
            pbar.update(1)


class FineTunedTestStep(EvaluationMethod):
    """
    Evaluation method for the f rozen base CLIP model.
    """

    @torch.no_grad()
    def evaluate(self, dataset, classnames: list[int], desc_add="") -> Dict[str, float]:
        self.model.eval()
        accuracy_meter = AverageMeter()
        tmp_dataset = ContiguousLabelDataset(dataset, classnames)
        dataloader = DataLoader(tmp_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)
        remapped_class_names = [CLASS_NAMES[tmp_dataset.idx2cat[i]] for i in range(len(tmp_dataset.idx2cat))]
        with self.model.temporary_classnames(remapped_class_names):
            pbar = tqdm(total=len(dataloader), desc="Test (Finetuned) " + desc_add, position=1, leave=False)
            for images, targets in dataloader:
                images = images.to(self.device)
                targets = targets.to(self.device)
                logits = self.model(images)
                predictions = logits.argmax(dim=-1)
                correct = (predictions == targets).sum().item()
                accuracy_meter.update(correct, n=targets.size(0), raw=True)
                pbar.set_postfix({
                    "accuracy": accuracy_meter.avg
                })
                pbar.update(1)

        return {"accuracy": accuracy_meter.avg}

In [14]:
# From: ./training_systems/evaluation_methods/__init__.py
# from training_systems.evaluation_methods.TestSteps import FineTunedTestStep, ZeroShotTestStep
# from training_systems.evaluation_methods.EvalStep import EvalStep

In [15]:
# From: ./training_systems/training_methods/Adversarial.py
"""
This module implements the Adversarial training method, which incorporates a gradient reversal layer
and an adversarial MLP to encourage domain-invariant feature learning.
"""
from typing import Dict, Any, List

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

# from training_systems.core import TrainingMethod
# from utils import AverageMeter, ContiguousLabelDataset, CLASS_NAMES
# from model.cocoop.mlp_adversary import AdversarialMLP, GradientReversalLayer


class Adversarial(TrainingMethod):
    """
    Adversarial training method using a Gradient Reversal Layer and an MLP adversary.

    Attributes:
        cls_cluster_dict (Dict[int, Any]): Maps class indices to cluster labels.
        grl (GradientReversalLayer): The gradient reversal layer instance.
        mlp_adversary (AdversarialMLP): The adversarial MLP used to confuse cluster prediction.
        lambda_adv (float): Weight of the adversarial loss term.
        debug (bool): If True, print debug information.
    """

    def __init__(
            self,
            model: Any,
            optimizer: Any,
            cls_cluster_dict: Dict[int, Any],
            grl: GradientReversalLayer,
            mlp_adversary: AdversarialMLP,
            lambda_adv,
            tmp_classes: list,
            debug: bool = False,
            gaussian_noise: float = 0.0,
            use_bias_ctx: bool = False
    ) -> None:
        """
        Args:
            model (Any): The main model being trained.
            optimizer (Any): Optimizer for updating model parameters.
            cls_cluster_dict (Dict[int, Any]): Mapping from class labels to clusters.
            grl (GradientReversalLayer): The gradient reversal layer.
            mlp_adversary (AdversarialMLP): The adversarial network module.
            lambda_adv (float): Weight for the adversarial loss.
            debug (bool, optional): Enables debug mode. Defaults to False.
        """
        super().__init__(model, optimizer, "Adv.", debug)
        self.cls_cluster_dict = cls_cluster_dict
        self.grl = grl
        self.mlp_adversary = mlp_adversary
        self.lambda_adv = lambda_adv
        self.tmp_classes = tmp_classes
        self.gaussian_noise = gaussian_noise
        self.use_bias_ctx = use_bias_ctx

    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Returns:
            Dict[str, AverageMeter]: Dictionary containing initialized metrics for training.
        """
        return {
            "total_loss_metric": AverageMeter(),
            "ce_loss_metric": AverageMeter(),
            "adv_loss_metric": AverageMeter(),
            "accuracy_metric": AverageMeter(),
        }

    def get_data_loader(self, dataset: ContiguousLabelDataset, batch_size: int) -> DataLoader:
        """
        Args:
            dataset (ContiguousLabelDataset): Dataset to be used.
            batch_size (int): Size of each batch.

        Returns:
            DataLoader: Configured PyTorch DataLoader instance.
        """
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
        )

    def forward_backward(
            self,
            sample,
            batch_idx,
            metrics: Dict[str, AverageMeter],
            dataset: ContiguousLabelDataset,
            accumulation_steps: int = 1,
            step: int = 0
    ) -> Dict[str, float]:
        """
        Executes the forward and backward pass.

        Args:
            sample (Tuple[Tensor, Tensor]): Batch of input data and targets.
            batch_idx (int): Index of the current batch.
            metrics (Dict[str, AverageMeter]): Dictionary of metrics to be updated.
            dataset (ContiguousLabelDataset): Dataset for cluster label lookup.

        Returns:
            Dict[str, float]: Dictionary with loss and accuracy metrics.
        """
        # Load data into GPU
        inputs, targets = sample
        inputs, targets = inputs.to(self.device), targets.to(self.device)

        targets_real_category = [dataset.idx2cat[c.item()] for c in targets]
        cluster_target = [int(self.cls_cluster_dict[int(tl)]) for tl in targets_real_category]
        cluster_target = torch.tensor(
            cluster_target,
            device=targets.device,
            dtype=torch.float32
        )

        # Use all tmp_classes for adversarial phase
        with self.model.temporary_classnames([CLASS_NAMES[idx] for idx in self.tmp_classes]):
            logits, ce_loss, img_features, ctx, bias, avg_txt_features, selected_text_features = self.model(inputs, targets, get_image_features=True)

            if self.gaussian_noise > 0:
                noise = torch.randn_like(logits) * self.gaussian_noise
                noisy_logits = logits + noise
            else:
                noisy_logits = logits

            if self.use_bias_ctx:
                if ctx.shape[0] == 1:
                    ctx = ctx.expand(bias.shape[0], -1, -1)

                ctx_shifted = ctx + bias  # shape: [B, L, D]
                # concat = ctx_shifted.view(ctx_shifted.size(0), -1).to(dtype=torch.float32)
                concat = torch.cat([selected_text_features, ctx_shifted.mean(dim=1)], dim=1).to(dtype=torch.float32)

            else:
                concat = torch.cat([avg_txt_features, noisy_logits], dim=1).to(dtype=torch.float32)
            # print(f"concat shape: {concat.shape}, bias shape: {bias.shape}, ctx shape: {ctx.shape}, avg_txt_features shape: {avg_txt_features.shape}, logits shape: {logits.shape}")
            reversed_concat = self.grl(concat)
            # print(f"reversed_concat shape: {reversed_concat.shape}")
            cluster_logits = self.mlp_adversary(reversed_concat)

            if cluster_logits.shape[1] == 1:
                # Binary classification
                cluster_logits = cluster_logits.squeeze(-1)
                loss_adv = F.binary_cross_entropy_with_logits(cluster_logits, cluster_target.float())
            else:
                # Multi-class classification
                loss_adv = F.cross_entropy(cluster_logits, cluster_target.long())

            # Skip adversarial update if prompt learner is frozen
            if any(p.requires_grad for p in self.model.prompt_learner.parameters()):
                ce_grads = self.get_grads(ce_loss)
            else:
                ce_grads = None  # Or torch.zeros_like(...), depending on downstream use
            total_loss = ce_loss + self.lambda_adv * loss_adv

            total_loss = total_loss / accumulation_steps
            total_loss.backward()
            # print(f"step: {step}, total_loss: {total_loss.item():.4f}, accumulation_steps: {accumulation_steps}, ")
            # --- accumulate grads and update ---
            if (step + 1) % accumulation_steps == 0: 
                torch.nn.utils.clip_grad_norm_(
                    list(self.model.parameters()) + list(self.mlp_adversary.parameters()),
                    max_norm=1.0,
                    norm_type=2.0,
                    error_if_nonfinite=True
                )
                self.optimizer_step()
                self.optimizer.zero_grad()
            
            if batch_idx < 3 and self.debug:  # Log only a few batches for performance
                print(f"[Batch {batch_idx}] CE Loss: {ce_loss.item():.4f} | "
                      f"Adv Loss: {loss_adv.item():.4f} | "
                      f"Total Loss: {total_loss.item():.4f} | "
                      f"lambda_adv: {self.lambda_adv:.4f}")
            batch_size = inputs.shape[0]
            metrics["total_loss_metric"].update(total_loss.item(), n=batch_size)
            metrics["ce_loss_metric"].update(ce_loss.item(), n=batch_size)
            metrics["adv_loss_metric"].update(loss_adv.item(), n=batch_size)
            _, predicted = logits.max(dim=1)
            correct = predicted.eq(targets).sum().item()
            metrics["accuracy_metric"].update(correct, n=batch_size, raw=True)
            return {
                "total_loss": total_loss.item(),
                "ce_loss": ce_loss.item(),
                "adv_loss": loss_adv.item(),
                "accuracy": correct / batch_size,
            }

    def print_grads_norms(self, bce_grads, ce_grads):
        """
        Args:
            bce_grads (Dict[str, Tensor]): Gradients from BCE loss.
            ce_grads (Dict[str, Tensor]): Gradients from CE loss.
        """
        for name in ce_grads:
            ce_norm = ce_grads[name].norm().item()
            bce_norm = bce_grads[name].norm().item()
            print(f"{name}: CE grad norm = {ce_norm:.4e}, BCE grad norm = {bce_norm:.4e}")

    def get_grads(self, loss):
        """
        Args:
            loss (Tensor): Loss tensor to backpropagate.

        Returns:
            Dict[str, Tensor]: Dictionary of gradients.
        """
        loss.backward(retain_graph=True)
        ce_grads = {}
        for name, p in self.model.named_parameters():
            if p.grad is not None and "prompt_learner" in name:
                ce_grads[name] = p.grad.detach().clone()
        # --- Zero gradients ---
        self.optimizer.zero_grad()
        return ce_grads

    def debug_metrics_to_pbar_args(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Args:
            debug_metrics (Dict[str, float]): Metrics from current training step.

        Returns:
            Dict[str, float]: Same metrics, passed to progress bar.
        """
        return debug_metrics

    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Args:
            metrics (Dict[str, AverageMeter]): Collected training metrics.

        Returns:
            List[float]: Average values of total, accuracy, CE, and adversarial losses.
        """
        return [
            metrics["total_loss_metric"].avg,
            metrics["accuracy_metric"].avg,
            metrics["ce_loss_metric"].avg,
            metrics["adv_loss_metric"].avg,
        ]

    def update_lambda_adv(self, lambda_adv) -> None:
        """
        Args:
            lambda_adv (float): New value to set for lambda_adv.
        """
        self.lambda_adv = lambda_adv

In [16]:
# From: ./training_systems/training_methods/BaseCoCoOp.py
"""
This module defines the BaseCoCoOp training method, a baseline implementation for CoCoOp-based optimization.
It includes metric tracking, data loading, and training step execution using standard cross-entropy loss.
"""
import random
from typing import Dict, Any

import torch
from torch.utils.data import DataLoader

# from training_systems.core import TrainingMethod
# from utils import AverageMeter, ContiguousLabelDataset


class BaseCoCoOp(TrainingMethod):
    """
    BaseCoCoOp implements a simple training routine based on cross-entropy loss without adversarial components.
    This class inherits from TrainingMethod and provides basic training loop functionalities.
    """

    def __init__(
            self,
            model: Any,
            optimizer: Any,
            debug: bool = False
    ) -> None:
        super().__init__(model, optimizer, "Base CoCoOp", debug)

    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Initializes and returns the performance metrics used during training.

        Returns:
            Dict[str, AverageMeter]: A dictionary with average meters for loss and accuracy.
        """


        return {
            "loss_metric": AverageMeter(),
            "accuracy_metric": AverageMeter(),
        }

    def get_data_loader(self, dataset: ContiguousLabelDataset, batch_size: int) -> DataLoader:
        """
        Creates and returns a DataLoader for the given dataset.

        Args:
            dataset (ContiguousLabelDataset): Dataset to load samples from.
            batch_size (int): Number of samples per batch.

        Returns:
            DataLoader: PyTorch DataLoader configured with shuffle and multiple workers.
        """
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
        )

    def forward_backward(
            self,
            sample,
            batch_idx,
            metrics: Dict[str, AverageMeter],
            dataset: ContiguousLabelDataset,
            accumulation_steps: int = 1,
            step: int = 0
    ) -> Dict[str, float]:
        """
        Executes the forward and backward pass for the BaseCoCoOp training method.

        Args:
            sample (Tuple[Tensor, Tensor]): Batch of input data and targets.
            batch_idx (int): Index of the current batch.
            metrics (Dict[str, AverageMeter]): Dictionary of metrics to be updated.
            dataset (ContiguousLabelDataset): Dataset object (unused in this implementation).

        Returns:
            Dict[str, float]: Dictionary with current loss and accuracy values.
        """
        # Load data into GPU
        inputs, targets = sample
        # === Pseudo-base: cross-entropy ===
        inputs_base = inputs.to(self.device)
        targets_base = targets.to(self.device)

        logits_base, loss_ce = self.model(inputs_base, targets_base)
        # === Combine losses ===
        print("SHAPES LOGITS: ",logits_base.shape, targets_base.shape)
        
        loss_ce = loss_ce / accumulation_steps
        loss_ce.backward()

        # optimizer step every `accumulation_steps` steps
        if (step + 1) % accumulation_steps == 0:
            self.optimizer.step()
            self.optimizer.zero_grad()

        self.optimizer_step()
        batch_size_total = inputs_base.size(0)

        metrics["loss_metric"].update(loss_ce.item(), n=batch_size_total)

        _, predicted = logits_base.max(dim=1)
        correct = (predicted == targets_base).sum().item()
        total = targets_base.size(0)
        metrics["accuracy_metric"].update(correct, n=total, raw=True)

        return {
            "loss": loss_ce.item(),
            "accuracy": correct / targets_base.size(0),
        }

    def debug_metrics_to_pbar_args(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Passes debug metrics directly to be displayed in the progress bar.

        Args:
            debug_metrics (Dict[str, float]): Metrics from the current step.

        Returns:
            Dict[str, float]: Unmodified metrics suitable for display.
        """
        return debug_metrics

    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Extracts and returns the average loss and accuracy metrics after a training step.

        Args:
            metrics (Dict[str, AverageMeter]): Dictionary containing tracked metrics.

        Returns:
            List[float]: A list containing the average loss and accuracy values.
        """
        return [
            metrics["loss_metric"].avg,
            metrics["accuracy_metric"].avg,
        ]






In [17]:
# From: ./training_systems/training_methods/KLCoCoOp.py
"""
This module defines the KLCoCoOp training method, which combines standard cross-entropy loss with KL divergence
between a model's predictions and a frozen teacher (e.g., CLIP) to enhance generalization to novel classes.
"""
import random
from typing import Dict, Any

import torch
from torch.utils.data import DataLoader

from training_systems.core.TrainingMethod import TrainingMethod

# from utils import AverageMeter,ContiguousLabelDataset, get_kl_loss
# from utils.datasets import CLASS_NAMES


class KLCoCoOp(TrainingMethod):
    """
    KLCoCoOp training method combining cross-entropy classification on pseudo-base samples and KL divergence
    loss on pseudo-novel samples. It encourages transferability and generalization by mixing base and novel categories.

    Attributes:
        lambda_kl (float): Weight for the KL divergence loss component.
    """

    def __init__(
            self,
            model: Any,
            optimizer: Any,
            lambda_kl,
            debug: bool = False
    ) -> None:
        super().__init__(model, optimizer, "Base CoCoOp + KL", debug)
        self.lambda_kl = lambda_kl

    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Initializes training metrics including total, cross-entropy, KL divergence losses, and classification accuracy.

        Returns:
            Dict[str, AverageMeter]: Dictionary of metric names mapped to their respective AverageMeter instances.
        """
        return {
            "total_loss_metric": AverageMeter(),
            "ce_loss_metric": AverageMeter(),
            "kl_loss_metric": AverageMeter(),
            "accuracy_metric": AverageMeter(),
        }

    def get_data_loader(self, dataset: ContiguousLabelDataset, batch_size: int) -> DataLoader:
        """
        Returns a DataLoader that splits each batch into pseudo-base and pseudo-novel subsets.

        Args:
            dataset (ContiguousLabelDataset): The dataset used for training.
            batch_size (int): Number of samples per batch.

        Returns:
            DataLoader: PyTorch DataLoader with a custom collate function to separate base and novel samples.
        """

        def custom_collate(batch):
            base_samples = []
            novel_samples = []
            targets_in_batch = list(set([target for _, target in batch]))
            random.shuffle(targets_in_batch)
            split_idx = int(0.7 * len(targets_in_batch))
            pseudo_base_ids = targets_in_batch[:split_idx]
            pseudo_novel_ids = targets_in_batch[split_idx:]
            for img, label in batch:
                if label in pseudo_base_ids:
                    base_samples.append((img, label))
                elif label in pseudo_novel_ids:
                    novel_samples.append((img, label))
            return base_samples, novel_samples

        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            collate_fn=custom_collate,
        )

    def forward_backward(
            self,
            sample,
            batch_idx,
            metrics: Dict[str, AverageMeter],
            dataset: ContiguousLabelDataset,
            accumulation_steps: int = 1,
            step: int = 0
    ) -> Dict[str, float]:
        """
        Performs forward and backward passes, computing CE loss on pseudo-base and KL loss on pseudo-novel samples.

        Args:
            sample (Tuple[List[Tuple[Tensor, int]], List[Tuple[Tensor, int]]]): Tuple of pseudo-base and pseudo-novel batches.
            batch_idx (int): Current batch index during training.
            metrics (Dict[str, AverageMeter]): Dictionary to update with the training metrics.
            dataset (ContiguousLabelDataset): Dataset object used for KL divergence lookup.

        Returns:
            Dict[str, float]: Scalar values of total loss, CE loss, KL loss, and CE accuracy.
        """
        # Load data into GPU
        base_batch, novel_batch = sample

        if not base_batch or not novel_batch:
            return {
                metric: val.avg
                for metric, val in metrics.items()
            }

        # === Pseudo-base: cross-entropy ===
        inputs_base = torch.stack([img for img, _ in base_batch]).to(self.device)
        targets_base = torch.tensor([lbl for _, lbl in base_batch]).to(self.device)


        categories_base_tensor = [dataset.idx2cat[int(c.item())] for c in list(set(targets_base))]
        remapped_class_names = [ CLASS_NAMES[ c ] for c in categories_base_tensor ]

        target_remapping = {cat:idx for idx, cat in enumerate(categories_base_tensor)}
        target_original = [dataset.idx2cat[int(c.item())] for c in targets_base]
        target_remapped = torch.tensor([target_remapping[c] for c in target_original]).to(self.device)

        with self.model.temporary_classnames(remapped_class_names):
            self.model.train()
            logits_base, loss_ce = self.model(inputs_base, target_remapped)

        # === Pseudo-novel: KL divergence with frozen CLIP ===
        self.model.eval()  # needed to disable dropout etc.
        inputs_novel = torch.stack([img for img, _ in novel_batch]).to(self.device)
        targets_novel = [lbl for _, lbl in novel_batch]

        kl_loss = get_kl_loss(self.device, inputs_novel, self.model, targets_novel, dataset)

        # === Combine losses ===
        total_loss = loss_ce + self.lambda_kl * kl_loss

        total_loss = total_loss / accumulation_steps
        total_loss.backward()

        # optimizer step every `accumulation_steps` steps
        if (step + 1) % accumulation_steps == 0:
            self.optimizer.step()
            self.optimizer.zero_grad()

        batch_size_total = inputs_base.size(0) + inputs_novel.size(0)

        metrics["total_loss_metric"].update(total_loss.item(), n=batch_size_total)
        metrics["ce_loss_metric"].update(loss_ce.item(), n=inputs_base.size(0))
        metrics["kl_loss_metric"].update(kl_loss.item(), n=inputs_novel.size(0))

        _, predicted = logits_base.max(dim=1)
        correct = (predicted == target_remapped).sum().item()
        total = target_remapped.size(0)
        metrics["accuracy_metric"].update(correct, n=total, raw=True)

        return {
            "total_loss": total_loss.item(),
            "ce_loss": loss_ce.item(),
            "ce_accuracy": correct / target_remapped.size(0),
            "kl_loss": kl_loss.item(),
        }

    def debug_metrics_to_pbar_args(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Prepares debug metrics for visualization in progress bars or logs.

        Args:
            debug_metrics (Dict[str, float]): Dictionary of debug metrics from the current step.

        Returns:
            Dict[str, float]: Same metrics for direct display.
        """
        return debug_metrics

    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Returns the average values of tracked metrics after a training step.

        Args:
            metrics (Dict[str, AverageMeter]): Metric dictionary to extract averages from.

        Returns:
            List[float]: Averages of total loss, accuracy, CE loss, and KL loss.
        """
        return [
            metrics["total_loss_metric"].avg,
            metrics["accuracy_metric"].avg,
            metrics["ce_loss_metric"].avg,
            metrics["kl_loss_metric"].avg,
        ]

    def update_lambda_kl(self, lambda_kl):
        self.lambda_kl = lambda_kl





In [18]:
# From: ./training_systems/training_methods/KLCoCoOpV2.py
"""
This module defines the KLCoCoOp training method, which combines standard cross-entropy loss with KL divergence
between a model's predictions and a frozen teacher (e.g., CLIP) to enhance generalization to novel classes.
"""
import random
from typing import Dict, Any, Optional

import clip
import torch
from torch.utils.data import DataLoader, Dataset

# from training_systems.core import DoubleDatasetTrainingMethod

# from utils import AverageMeter,CLASS_NAMES, ContiguousLabelDataset, get_kl_loss


class KLCoCoOpV2(DoubleDatasetTrainingMethod):
    """
    KLCoCoOp training method combining cross-entropy classification on pseudo-base samples and KL divergence
    loss on pseudo-novel samples. It encourages transferability and generalization by mixing base and novel categories.

    Attributes:
        lambda_kl (float): Weight for the KL divergence loss component.
    """

    def __init__(
            self,
            model: Any,
            optimizer: Any,
            lambda_kl,
            debug: bool = False,
    ) -> None:
        super().__init__(model, optimizer, "Base CoCoOp + KL", debug)
        self.lambda_kl = lambda_kl
        if self.debug:
            print(f"[KLCoCoOpV2] Initialized with lambda_kl={self.lambda_kl}, device={self.device}")


    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Initializes training metrics including total, cross-entropy, KL divergence losses, and classification accuracy.

        Returns:
            Dict[str, AverageMeter]: Dictionary of metric names mapped to their respective AverageMeter instances.
        """
        if self.debug:
            print("[KLCoCoOpV2] Initializing metrics.")
        return {
            "ce_loss_metric": AverageMeter(),   
            "kl_loss_metric": AverageMeter(),
            "ce_accuracy_metric": AverageMeter(),
        }

    def get_data_loader1(self, pseudo_base: ContiguousLabelDataset, batch_size: int) -> DataLoader:
        """
        Returns a DataLoader that splits each batch into pseudo-base and pseudo-novel subsets.

        Args:
            dataset (ContiguousLabelDataset): The dataset used for training.
            batch_size (int): Number of samples per batch.

        Returns:
            DataLoader: PyTorch DataLoader with a custom collate function to separate base and novel samples.
        """
        if self.debug:
            print(f"[KLCoCoOpV2] Creating DataLoader1 for pseudo_base with batch_size={batch_size}.")
        return DataLoader(
            pseudo_base,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
        )

    def get_data_loader2(self, pseudo_novel_dataset: ContiguousLabelDataset, batch_size: int) -> DataLoader:
        """
        Returns a DataLoader that splits each batch into pseudo-base and pseudo-novel subsets.
        """
        if self.debug:
            print(f"[KLCoCoOpV2] Creating DataLoader2 for pseudo_novel with batch_size={batch_size}.")
        return DataLoader(pseudo_novel_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    
    def forward_backward1(
            self,
            sample,
            batch_idx,
            metrics: Dict[str, AverageMeter],
            dataset: ContiguousLabelDataset,
            classes: list[int]
    ) -> Dict[str, float]:
        if self.debug and batch_idx < 3:
            print(f"[KLCoCoOpV2] forward_backward1: batch_idx={batch_idx}")
            print(f"[KLCoCoOpV2] Sample type: {type(sample)}")
        # Load data into GPU
        inputs, targets = sample
        # === Pseudo-base: cross-entropy ===
        inputs_base = inputs.to(self.device)
        targets_base = targets.to(self.device)
        # Use remapped class names to match the remapped labels
        # classes contains original category indices, but we need to map them to 0-based indices
        # The ContiguousLabelDataset remaps labels to 0-based indices, so we need class names in the same order
        # Use the remapped indices to get class names in the correct order
        remapped_class_names = [ CLASS_NAMES[ dataset.idx2cat[i] ] for i in range(len(dataset.idx2cat)) ]
        assert set(classes) == set(dataset.idx2cat.values()), \
            f"Split classes ({classes}) != dataset labels ({list(dataset.idx2cat.values())})"
        with self.model.temporary_classnames(remapped_class_names):
            logits_base, loss_ce = self.model(inputs_base, targets_base)
            # === Combine losses ===
            if self.debug and batch_idx < 3:
                print(f"[KLCoCoOpV2] LOGITS shape: {logits_base.shape}, TARGETS shape: {targets_base.shape}")
                print(f"[KLCoCoOpV2] CE loss: {loss_ce.item()}")
                print(f"[KLCoCoOpV2] Remapped class names: {remapped_class_names}")
                print(f"[KLCoCoOpV2] Original classes: {classes}")
                print(f"[KLCoCoOpV2] Targets: {targets_base}")
                print(f"[KLCoCoOpV2] Logits max: {logits_base.max(dim=1)[1]}")
                print(f"[KLCoCoOpV2] Dataset cat2idx: {dataset.cat2idx}")
                print(f"[KLCoCoOpV2] Dataset idx2cat: {dataset.idx2cat}")
                print(f"[KLCoCoOpV2] Expected class names order: {[CLASS_NAMES[dataset.idx2cat[i]] for i in range(len(classes))]}")
                print(f"[KLCoCoOpV2] Model class names: {[CLASS_NAMES[c] for c in classes]}")
            loss_ce.backward()

            self.optimizer_step()
            batch_size_total = inputs_base.size(0)

            metrics["ce_loss_metric"].update(loss_ce.item(), n=batch_size_total)

            _, predicted = logits_base.max(dim=1)
            correct = (predicted == targets_base).sum().item()
            total = targets_base.size(0)
            metrics["ce_accuracy_metric"].update(correct, n=total, raw=True)

            if self.debug and batch_idx < 3:
                print(f"[KLCoCoOpV2] Batch accuracy: {correct}/{total} = {correct/total if total > 0 else 0}")
        return {
            "ce_loss": loss_ce.item(),
            "accuracy": correct / targets_base.size(0),
        }

    def forward_backward2(
            self,
            sample,
            batch_idx,
            metrics: Dict[str, AverageMeter],
            dataset: ContiguousLabelDataset,
            classes: list[int]
    ) -> Dict[str, float]:
        # Use remapped class names to match the remapped labels
        # classes contains original category indices, but we need to map them to 0-based indices
        # The ContiguousLabelDataset remaps labels to 0-based indices, so we need class names in the same order
        # Use the remapped indices to get class names in the correct order
        remapped_class_names = [ CLASS_NAMES[ dataset.idx2cat[i] ] for i in range(len(dataset.idx2cat)) ]
        pseudo_novel_class_names = remapped_class_names
        if self.debug and batch_idx < 3:
            print(f"[KLCoCoOpV2] forward_backward2: batch_idx={batch_idx}")
            print(f"[KLCoCoOpV2] Sample type: {type(sample)}; Sample len: {len(sample) if hasattr(sample, '__len__') else 'N/A'}")
        # Load data into GPU
        inputs_novel, targets_novel = sample
        # === Pseudo-novel: KL divergence with frozen CLIP ===
        inputs_novel = inputs_novel.to(self.device)
        targets_novel = targets_novel.to(self.device)

        with torch.no_grad():
            image_features_clip = self.model.clip_model.encode_image(inputs_novel)
            image_features_clip = image_features_clip / image_features_clip.norm(dim=-1, keepdim=True)

            #category_idxs = [dataset.idx2cat[c.item()] for c in list(set(targets_novel))] # type: ignore

            text_inputs = clip.tokenize(
                [f"a photo of a {cn}, a type of flower." for cn in pseudo_novel_class_names]
            ).to(self.device)

            text_features_clip = self.model.clip_model.encode_text(text_inputs)
            text_features_clip = text_features_clip / text_features_clip.norm(dim=-1, keepdim=True)

            clip_logits = image_features_clip @ text_features_clip.T


        self.model.train()
        with self.model.temporary_classnames(remapped_class_names):
            student_logits, student_loss = self.model(inputs_novel, targets_novel) 
            kl_loss = torch.nn.functional.kl_div(
                torch.nn.functional.log_softmax(student_logits, dim=-1),
                torch.nn.functional.softmax(clip_logits, dim=-1),
                reduction="batchmean"
            )

        # === Combine losses ===
            kl_loss = self.lambda_kl * kl_loss

            if self.debug and batch_idx < 3:
                print(f"[KLCoCoOpV2] KL loss (weighted): {kl_loss.item()}")
            kl_loss.backward()

            self.optimizer_step()

            metrics["kl_loss_metric"].update(kl_loss.item(), n=inputs_novel.size(0))

        return {
            "kl_loss": kl_loss.item(),
        }

    def debug_metrics_to_pbar_args1(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Prepares debug metrics for visualization in progress bars or logs.

        Args:
            debug_metrics (Dict[str, float]): Dictionary of debug metrics from the current step.

        Returns:
            Dict[str, float]: Same metrics for direct display.
        """
        if self.debug:
            print(f"[KLCoCoOpV2] debug_metrics_to_pbar_args1: {debug_metrics}")
        return debug_metrics

    def debug_metrics_to_pbar_args2(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Prepares debug metrics for visualization in progress bars or logs.
        """
        if self.debug:
            print(f"[KLCoCoOpV2] debug_metrics_to_pbar_args2: {debug_metrics}")
        return debug_metrics

    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Returns the average values of tracked metrics after a training step.

        Args:
            metrics (Dict[str, AverageMeter]): Metric dictionary to extract averages from.

        Returns:
            List[float]: Averages of total loss, accuracy, CE loss, and KL loss.
        """
        if self.debug:
            print(f"[KLCoCoOpV2] training_step_return: KL={metrics['kl_loss_metric'].avg}, CE={metrics['ce_loss_metric'].avg}, Acc={metrics['ce_accuracy_metric'].avg}")
        return [
            metrics["kl_loss_metric"].avg,
            metrics["ce_loss_metric"].avg,
            metrics["ce_accuracy_metric"].avg,
        ]

    def update_lambda_kl(self, lambda_kl):
        """
        Update the lambda_kl value used for KL loss weighting.
        
        Args:
            lambda_kl (float): New lambda_kl value.
        """
        self.lambda_kl = lambda_kl






In [19]:
# From: ./training_systems/training_methods/OnlyMLP.py
"""
This module implements the Adversarial training method, which incorporates a gradient reversal layer
and an adversarial MLP to encourage domain-invariant feature learning.
"""

from typing import Dict, Any

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

# from training_systems.core import TrainingMethod
# from utils import AverageMeter, ContiguousLabelDataset
# from model.cocoop.mlp_adversary import AdversarialMLP, GradientReversalLayer


class OnlyMLP(TrainingMethod):
    """
    Adversarial training method using a Gradient Reversal Layer and an MLP adversary.

    Attributes:
        cls_cluster_dict (Dict[int, Any]): Maps class indices to cluster labels.
        grl (GradientReversalLayer): The gradient reversal layer instance.
        mlp_adversary (AdversarialMLP): The adversarial MLP used to confuse cluster prediction.
        lambda_adv (float): Weight of the adversarial loss term.
        debug (bool): If True, print debug information.
    """

    def __init__(
            self,
            model: Any,
            optimizer: Any,
            cls_cluster_dict: Dict[int, Any],
            mlp_adversary: AdversarialMLP,
            debug: bool = False
    ) -> None:
        """
        Args:
            model (Any): The main model being trained.
            optimizer (Any): Optimizer for updating model parameters.
            cls_cluster_dict (Dict[int, Any]): Mapping from class labels to clusters.
            grl (GradientReversalLayer): The gradient reversal layer.
            mlp_adversary (AdversarialMLP): The adversarial network module.
            lambda_adv (float): Weight for the adversarial loss.
            debug (bool, optional): Enables debug mode. Defaults to False.
        """
        super().__init__(model, optimizer, "Adv.", debug)
        self.cls_cluster_dict = cls_cluster_dict
        self.mlp_adversary = mlp_adversary

    def get_metrics(self) -> Dict[str, AverageMeter]:
        """
        Returns:
            Dict[str, AverageMeter]: Dictionary containing initialized metrics for training.
        """
        return {
            "adv_loss_metric": AverageMeter(),
            "accuracy_metric": AverageMeter(),
        }

    def get_data_loader(self, dataset: ContiguousLabelDataset, batch_size: int) -> DataLoader:
        """
        Args:
            dataset (ContiguousLabelDataset): Dataset to be used.
            batch_size (int): Size of each batch.

        Returns:
            DataLoader: Configured PyTorch DataLoader instance.
        """
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
        )

    def forward_backward(
            self,
            sample,
            batch_idx,
            metrics: Dict[str, AverageMeter],
            dataset: ContiguousLabelDataset,
            accumulation_steps: int = 1,
            step: int = 0
    ) -> Dict[str, float]:
        """
        Executes the forward and backward pass.

        Args:
            sample (Tuple[Tensor, Tensor]): Batch of input data and targets.
            batch_idx (int): Index of the current batch.
            metrics (Dict[str, AverageMeter]): Dictionary of metrics to be updated.
            dataset (ContiguousLabelDataset): Dataset for cluster label lookup.

        Returns:
            Dict[str, float]: Dictionary with loss and accuracy metrics.
        """
        # Load data into GPU
        inputs, targets = sample
        inputs, targets = inputs.to(self.device), targets.to(self.device)

        targets_real_category = [dataset.idx2cat[c.item()] for c in targets]
        cluster_target = [int(self.cls_cluster_dict[int(tl)]) for tl in targets_real_category]
        cluster_target = torch.tensor(
            cluster_target,
            device=targets.device,
            dtype=torch.float32
        )

        # Forward pass + loss computation dsada
        logits, ce_loss, img_features, ctx, bias = self.model(inputs, targets, get_image_features=True)
        ctx = ctx.detach()  # Detach context to avoid backprop through it
        ctx_shifted = ctx + bias.unsqueeze(1).detach()  # Add bias to context tokens
        ctx_flat = ctx_shifted.reshape(ctx_shifted.size(0), -1).to(dtype=torch.float32)

        cluster_logits = self.mlp_adversary(ctx_flat).squeeze()

        loss_bce = F.binary_cross_entropy_with_logits(cluster_logits, cluster_target)

        loss_bce = loss_bce / accumulation_steps
        # Backward pass
        loss_bce.backward()

        if (step + 1) % accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(
                list(self.model.parameters()) + list(self.mlp_adversary.parameters()),
                max_norm=1.0,
                norm_type=2.0,
                error_if_nonfinite=True
            )

            self.optimizer_step()
            self.optimizer.zero_grad()


        # Fetch prediction and loss value
        batch_size = inputs.shape[0]
        metrics["adv_loss_metric"].update(loss_bce.item(), n=batch_size)

        predicted = cluster_logits.gt(0.5).float()  # Convert logits to binary predictions
        correct = predicted.eq(cluster_target).sum().item()
        # Compute training accuracy
        metrics["accuracy_metric"].update(correct, n=batch_size, raw=True)

        return {
            "adv_loss": loss_bce.item(),
            "accuracy": correct / batch_size,
        }

    def debug_metrics_to_pbar_args(self, debug_metrics: Dict[str, float]) -> Dict[str, float]:
        """
        Args:
            debug_metrics (Dict[str, float]): Metrics from current training step.

        Returns:
            Dict[str, float]: Same metrics, passed to progress bar.
        """
        return debug_metrics

    def training_step_return(self, metrics: Dict[str, AverageMeter]) -> list[float]:
        """
        Args:
            metrics (Dict[str, AverageMeter]): Collected training metrics.

        Returns:
            List[float]: Average values of total, accuracy, CE, and adversarial losses.
        """
        return [
            metrics["accuracy_metric"].avg,
            metrics["adv_loss_metric"].avg,
        ]

    def update_lambda_adv(self, lambda_adv) -> None:
        """
        Args:
            lambda_adv (float): New value to set for lambda_adv.
        """
        self.lambda_adv = lambda_adv

In [20]:
# From: ./training_systems/training_methods/__init__.py
# from training_systems.training_methods.Adversarial import Adversarial
# from training_systems.training_methods.KLCoCoOp import KLCoCoOp
# from training_systems.training_methods.KLCoCoOpV2 import KLCoCoOpV2
# from training_systems.training_methods.BaseCoCoOp import BaseCoCoOp

In [21]:
# From: ./utils/__init__.py
# from utils.clustering import conditional_clustering, random_clustering, rotating_cluster_generator_shift
# from utils.datasets import (
#     get_data,
#     base_novel_categories,
#     split_data,
#     CLASS_NAMES,
#     ContiguousLabelDataset,
# )
# from utils.tensor_board_logger import TensorboardLogger
# from utils.kl import get_kl_loss
# from utils.metrics import AverageMeter

In [22]:
# From: ./utils/clustering.py
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tqdm import tqdm
import torch
# from utils.datasets import get_data, base_novel_categories, split_data, CLASS_NAMES
import clip
import os
import pickle
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_distances
from collections import Counter, deque


import random
from collections import deque

def create_cluster_from_ordered_list(ordered_categories, split_ratio):
    """
    Converts a list of categories into a cluster dict using split_ratio.
    """
    n = len(ordered_categories)
    n_zeros = int(n * split_ratio)
    return {
        cat: 0 if i < n_zeros else 1
        for i, cat in enumerate(ordered_categories)
    }

def rotating_cluster_generator_shift(categories, split_ratio, steps=1, seed=None):
    """
    Yields clusters by cyclically rotating the category list.

    Args:
        categories (list): List of category identifiers.
        split_ratio (float): Ratio of cluster 0 elements.
        seed (int, optional): Random seed for reproducibility.

    Yields:
        dict: Cluster mapping.
    """
    cat_list = list(categories)
    if seed is not None:
        random.seed(seed)
    random.shuffle(cat_list)
    cat_deque = deque(cat_list)

    while True:
        cluster = create_cluster_from_ordered_list(cat_deque, split_ratio)
        cat0 = [cat for cat, c in cluster.items() if c == 0]
        cat1 = [cat for cat, c in cluster.items() if c == 1]
        yield cluster, cat0, cat1
        cat_deque.rotate(-steps)  # rotate left

def cluster_categories(device, cnn, n_clusters=2, variance=0.95, data_dir="../data"):
    """
    Clusters base classes using visual features extracted from a CLIP model. Applies PCA to reduce dimensionality
    and Agglomerative Clustering on cosine distances to group the categories.

    Args:
        device (torch.device): The device to run computations on (CPU/GPU).
        cnn (str): CLIP model architecture name (e.g., "ViT-B/32").
        n_clusters (int): Number of clusters to generate.
        variance (float): Variance ratio to preserve during PCA.

    Returns:
        Tuple[Dict[int, int], Dict[str, int]]: Two dictionaries mapping class indices and class names to cluster IDs.
    """

    # initialize clip model with ViT
    clip_model, _ = clip.load(cnn)
    clip_model = clip_model.to(device)
    resolution = clip_model.visual.input_resolution
    train_set, _, _ = get_data(data_dir=data_dir, resolution=resolution)

    # split classes into base and novel
    base_classes, _ = base_novel_categories(train_set)

    # split the three datasets
    train_base, _ = split_data(train_set, base_classes)

    class_feature = {}
    with torch.no_grad():
        for c in tqdm(base_classes, desc="Processing classes"):
            imgs_c = []
            # Create a DataLoader to iterate over the dataset properly
            dataloader = torch.utils.data.DataLoader(train_base, batch_size=1, shuffle=False)
            for img, label in dataloader:
                if label.item() == c:
                    imgs_c.append(img.squeeze(0))
            features = [
                clip_model.encode_image(img.unsqueeze(0).to(device)).cpu().numpy()
                for img in imgs_c
            ]
            class_feature[c] = np.mean(features, axis=0)

    # class_ft_array = np.array([class_feature[c][0] for c in base_classes])

    cat2idx = {}
    idx2cat = {}
    class_ft_array = []
    for i, c in enumerate(base_classes):
        cat2idx[c] = i
        idx2cat[i] = c
        class_ft_array.append(class_feature[c][0])

    pca = PCA(n_components=variance)
    X_reduced = pca.fit_transform(class_ft_array)

    print(
        f"Reduced feature shape: {X_reduced.shape}, Variance explained: {pca.explained_variance_ratio_.sum()}"
    )

    cosine_dist = cosine_distances(X_reduced)
    # Step 5: Agglomerative clustering
    agglo = AgglomerativeClustering(
        n_clusters=n_clusters, metric="precomputed", linkage="average"
    )
    cluster_labels = agglo.fit_predict(cosine_dist)

    cluster_labels = {idx2cat[i]: cluster for i, cluster in enumerate(cluster_labels)}

    cluster_labels_text = {
        CLASS_NAMES[base_class]: int(cluster)
        for base_class, cluster in enumerate(cluster_labels)
    }

    torch.cuda.empty_cache()

    return cluster_labels, cluster_labels_text


def random_clustering(
    n_cluster,
    seed=42,
    data_dir="../data",
    distribution="uniform",
    split_ratio=0.7,  # Only for bipartite
):
    """
    Generates random cluster assignments for a given number of clusters.

    Args:
        n_cluster (int): Number of clusters to generate.
        seed (int): Random seed for reproducibility.
        data_dir (str): Directory where the dataset is stored.
        distribution (str): Distribution type for cluster assignment. Options are "uniform", "random", "sequential", or "bipartite".
        split_ratio (float): Percentage of classes in the larger cluster (only used for "bipartite").

    Returns:
        Tuple[
            Dict[int, int],        # class_id -> cluster_id
            Dict[str, int],        # class_name -> cluster_id
            List[int],             # class_ids in cluster 0
            List[int],             # class_ids in cluster 1
        ]
    """

    np.random.seed(seed)
    train_set, _, _ = get_data(data_dir=data_dir)
    base_classes, _ = base_novel_categories(train_set)

    cluster_labels = {}

    if distribution == "uniform":
        shuffled = np.random.permutation(base_classes)
        for i, cls in enumerate(shuffled):
            cluster_id = i % n_cluster
            cluster_labels[cls] = cluster_id

    elif distribution == "random":
        for cls in base_classes:
            cluster_id = np.random.choice(range(n_cluster))
            cluster_labels[cls] = cluster_id

    elif distribution == "sequential":
        for i, cls in enumerate(base_classes):
            cluster_id = i % n_cluster
            cluster_labels[cls] = cluster_id

    cluster_labels_text = {
        CLASS_NAMES[cls]: int(cluster_labels[cls])
        for cls in cluster_labels
    }

    cluster_dict_int = {int(k): v for k, v in cluster_labels.items()}

    return cluster_dict_int, cluster_labels_text


def conditional_clustering(n_cluster, variance, cnn, device, data_dir="../data"):
    """
    Loads existing cluster labels from disk if available, otherwise computes and saves new cluster assignments.

    Args:
        n_cluster (int): Number of clusters to generate.
        variance (float): Variance ratio to preserve during PCA.
        cnn (str): CLIP model architecture name (used for naming output files).
        device (torch.device): The device to run computations on.

    Returns:
        Tuple[Dict[int, int], Dict[str, int]]: Dictionaries for integer-labeled and text-labeled cluster assignments.
    """
    cnn_sanitized = cnn.replace("/", "_")
    save_dir = f"clustering_split/cluster_labels_{n_cluster}_{variance}_{cnn_sanitized}"
    os.makedirs(save_dir, exist_ok=True)

    int_categories_path = os.path.join(save_dir, "int_categories.pkl")
    text_categories_path = os.path.join(save_dir, "text_categories.pkl")


    # Commented this lines because they are not used in the Jupyter version, they are meant to save and load cluster labels

    # if os.path.exists(int_categories_path) and os.path.exists(text_categories_path):
    #     print("🟩 CLUSTERS FILES FOUND. Loading existing cluster labels...")
    #     with open(int_categories_path, "rb") as f:
    #         cluster_labels = pickle.load(f)
    #         cluster_dict_int = {int(k): v for k, v in cluster_labels.items()}
    #     with open(text_categories_path, "rb") as f:
    #         cluster_labels_text = pickle.load(f)

    # else:
    print("🟧 NO CLUSTERS FILES FOUND (jupyter version without saves). Loading existing cluster labels...")
    # cluster the base classes
    cluster_labels, cluster_labels_text = cluster_categories(
        device, n_clusters=n_cluster, variance=variance, cnn=cnn, data_dir=data_dir
    )
    cluster_dict_int = {int(k): v for k, v in cluster_labels.items()}
    # with open(int_categories_path, "wb") as f:
    #     pickle.dump(cluster_labels, f)
    # with open(text_categories_path, "wb") as f:
    #     pickle.dump(cluster_labels_text, f)
    # Count samples in each cluster
    cluster_counts = Counter(cluster_dict_int.values())
    for cluster_id in range(n_cluster):
        print(f"Cluster {cluster_id} count: {cluster_counts.get(cluster_id, 0)}")

    return cluster_dict_int, cluster_labels_text



In [23]:
# From: ./utils/datasets.py
import clip
import torch
import torchvision
from torch.utils.data import Subset
import torchvision.transforms as T

CLASS_NAMES = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid", "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris", "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary", "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation", "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip", "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy", "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia", "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum", "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory", "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow", "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]

# --- Data Augmentation Pipeline ---
def build_default_transform(resolution=224):
    """
    Builds the default data augmentation pipeline as specified:
    - RandomResizedCrop with bicubic interpolation
    - RandomHorizontalFlip
    - Normalize with given mean and std
    """
    return T.Compose([
        T.RandomResizedCrop(resolution, interpolation=T.InterpolationMode.BICUBIC),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
    ])


def build_eval_transform(resolution=224):
    """
    Builds the evaluation transform pipeline (no random augmentations):
    - Resize to resolution
    - CenterCrop to resolution
    - Normalize with given mean and std
    """
    return T.Compose([
        T.Resize(resolution, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(resolution),
        T.ToTensor(),
        T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
    ])


def get_data(data_dir="./data", train_transform=None, eval_transform=None, resolution=224):
    """
    Loads the Flowers102 dataset from torchvision, returning separate splits for training, validation, and testing.

    Args:
        data_dir (str): Directory where the dataset will be downloaded/stored. Defaults to "./data".
        train_transform (torchvision.transforms.Compose or None): Transformations to apply to training data.
        eval_transform (torchvision.transforms.Compose or None): Transformations to apply to validation/test data.
        resolution (int): Image resolution for transforms (default 224).

    Returns:
        tuple: A tuple (train, val, test) of Flowers102 dataset splits.
    """
    if train_transform is None:
        train_transform = build_default_transform(resolution)
    if eval_transform is None:
        eval_transform = build_eval_transform(resolution)
        
    train = torchvision.datasets.Flowers102(root=data_dir, split="train", download=True, transform=train_transform)
    val = torchvision.datasets.Flowers102(root=data_dir, split="val", download=True, transform=eval_transform)
    test = torchvision.datasets.Flowers102(root=data_dir, split="test", download=True, transform=eval_transform)
    return train, val, test


def base_novel_categories(dataset):
    # set returns the unique set of all dataset classes
    all_classes = set(dataset._labels)
    # and let's count them
    num_classes = len(all_classes)

    # here list(range(num_classes)) returns a list from 0 to num_classes - 1
    # then we slice the list in half and generate base and novel category lists
    base_classes = list(range(num_classes))[:num_classes//2]
    novel_classes = list(range(num_classes))[num_classes//2:]
    return base_classes, novel_classes


def get_labels(dataset):
    """
    Recursively retrieve labels from dataset or nested Subset.
    Assumes the base dataset has a `_labels` attribute.
    """
    if hasattr(dataset, '_labels'):
        return dataset._labels
    elif isinstance(dataset, Subset):
        parent_labels = get_labels(dataset.dataset)
        return [parent_labels[i] for i in dataset.indices]
    else:
        raise AttributeError("Dataset does not have _labels or is not a Subset of a dataset with _labels.")

def split_data(dataset, base_classes):
    """
    Splits the dataset into base and novel subsets based on base_classes.
    Works even if the input dataset is already a Subset.

    Args:
        dataset: PyTorch Dataset or Subset
        base_classes (List[int]): List of class indices considered as base.

    Returns:
        base_dataset (Subset): Subset containing samples from base classes.
        novel_dataset (Subset): Subset containing samples from novel classes.
    """
    base_categories_samples = []
    novel_categories_samples = []

    labels = get_labels(dataset)
    base_set = set(base_classes)

    for sample_id, label in enumerate(labels):
        if label in base_set:
            base_categories_samples.append(sample_id)
        else:
            novel_categories_samples.append(sample_id)

    base_dataset = Subset(dataset, base_categories_samples)
    novel_dataset = Subset(dataset, novel_categories_samples)

    return base_dataset, novel_dataset


class ContiguousLabelDataset(torch.utils.data.Dataset):
    """
    A dataset wrapper that remaps arbitrary class labels to contiguous integers starting from 0.

    This is useful for classification tasks where models expect class indices to be in a 0-based contiguous range.

    Attributes:
        dataset (Dataset): The original dataset to wrap.
        cat2idx (Dict[Any, int]): Mapping from original class labels to contiguous integer indices.
        idx2cat (Dict[int, Any]): Reverse mapping from indices back to original class labels.
    """
    def __init__(self, dataset, class_order: list[int]):
        self.dataset = dataset
        # force the mapping to use your custom order
        self.cat2idx = { cat: idx for idx, cat in enumerate(class_order) }
        self.idx2cat = { idx: cat for cat, idx in self.cat2idx.items() }

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        image, label = self.dataset[index]
        mapped_label = self.cat2idx[label]
        return image, mapped_label

# if __name__ == "__main__":
#     # Load model
#     clip_model, preprocess = clip.load("ViT-B/32", device="mps")
#     clip_model = clip_model.to("mps")

#     train_set, val_set, test_set = get_data(train_transform=preprocess, eval_transform=preprocess, data_dir="../data")
#     base_classes, novel_classes = base_novel_categories(train_set)
#     print(len(base_classes))

In [24]:
# From: ./utils/debugging.py
import torch


def check_gradients(model):
    print("=== Checking gradients ===")
    for name, param in model.named_parameters():
        if param.requires_grad:
            if param.grad is None:
                print(f"{name}: ❌ No gradient")
            elif torch.all(param.grad == 0):
                print(f"{name}: ⚠️ Zero gradient")
            else:
                print(f"{name}: ✅ Gradient flowing")




In [25]:
# From: ./utils/kl.py
import clip
import torch

# from utils.datasets import CLASS_NAMES


def get_kl_loss(device, inputs_novel, model, targets_novel, tmp_dataset):
    """
    Computes the KL divergence between the student model's predictions and the CLIP model's predictions
    for a batch of novel class images.

    Args:
        device (torch.device): The device (CPU or CUDA) to perform computation on.
        inputs_novel (Tensor): A batch of input images from novel classes.
        model (nn.Module): The student model that includes a CLIP backbone and prompt learner.
        targets_novel (List[int]): Target labels corresponding to the novel class inputs.
        tmp_dataset (ContiguousLabelDataset): Dataset wrapper with label-to-category mappings.

    Returns:
        Tensor: A scalar tensor representing the KL divergence loss.
    """
    targets_novel_tensor = torch.tensor(targets_novel).to(device) if isinstance(targets_novel, list) else targets_novel
    categories_novel_tensor = [tmp_dataset.idx2cat[c.item()] for c in list(set(targets_novel_tensor))]
    # print(f"input novel shape: {inputs_novel.shape} novel base: {targets_novel_tensor.shape}")
    with torch.no_grad():
        image_features_clip = model.clip_model.encode_image(inputs_novel)
        image_features_clip = image_features_clip / image_features_clip.norm(dim=-1, keepdim=True)


        text_inputs = clip.tokenize(
            [f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in categories_novel_tensor]
        ).to(device)

        text_features_clip = model.clip_model.encode_text(text_inputs)
        text_features_clip = text_features_clip / text_features_clip.norm(dim=-1, keepdim=True)

        clip_logits = image_features_clip @ text_features_clip.T

    remapped_class_names = [ CLASS_NAMES[ c ] for c in categories_novel_tensor ]
    
    # targets_novel_tensor contains contiguous indices (0, 1, 2, ...)
    # categories_novel_tensor should be the original class labels for the current batch
    
    # No need to remap targets; they are already correct
    target_remapping = {cat:idx for idx, cat in enumerate(categories_novel_tensor)}
    target_original = [tmp_dataset.idx2cat[c.item()] for c in targets_novel_tensor]
    target_remapped = torch.tensor([target_remapping[c] for c in target_original]).to(device)
    
    with model.temporary_classnames(remapped_class_names):
        model.train()
        student_logits, student_loss = model(inputs_novel, target_remapped)  # [B, num_classes]
        kl_loss = torch.nn.functional.kl_div(
            torch.nn.functional.log_softmax(student_logits, dim=-1),
            torch.nn.functional.softmax(clip_logits, dim=-1),
            reduction="batchmean"
        )
    return kl_loss

In [26]:
# From: ./utils/metrics.py
from collections import defaultdict

import torch


class AverageMeter:
    """Compute and store the average and current value.

    Examples::
        >>> # 1. Initialize a meter to record loss
        >>> losses = AverageMeter()
        >>> # 2. Update meter after every mini-batch update
        >>> losses.update(loss_value, batch_size)
    """

    def __init__(self, ema=False):
        """
        Args:
            ema (bool, optional): apply exponential moving average.
        """
        self.ema = ema
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1, raw=False):
        if isinstance(val, torch.Tensor):
            val = val.item()

        self.val = val
        if raw:
            self.sum += val
        else:
            self.sum += val * n

        self.count += n

        if self.ema:
            self.avg = self.avg * 0.9 + self.val * 0.1
        else:
            self.avg = self.sum / self.count


class MetricMeter:
    """Store the average and current value for a set of metrics.

    Examples::
        >>> # 1. Create an instance of MetricMeter
        >>> metric = MetricMeter()
        >>> # 2. Update using a dictionary as input
        >>> input_dict = {'loss_1': value_1, 'loss_2': value_2}
        >>> metric.update(input_dict)
        >>> # 3. Convert to string and print
        >>> print(str(metric))
    """

    def __init__(self, delimiter=" "):
        self.meters = defaultdict(AverageMeter)
        self.delimiter = delimiter

    def update(self, input_dict):
        if input_dict is None:
            return

        if not isinstance(input_dict, dict):
            raise TypeError(
                "Input to MetricMeter.update() must be a dictionary"
            )

        for k, v in input_dict.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            self.meters[k].update(v)

    def __str__(self):
        output_str = []
        for name, meter in self.meters.items():
            output_str.append(f"{name} {meter.val:.4f} ({meter.avg:.4f})")
        return self.delimiter.join(output_str)


if __name__ == "__main__":
    # Example usage
    metric = MetricMeter()
    metric.update({"loss_1": torch.tensor(0.5), "loss_2": torch.tensor(0.8)})
    metric.update({"loss_1": torch.tensor(0.4)})
    print(str(metric))
    metric.update({"loss_1": torch.tensor(0.6), "loss_2": torch.tensor(0.7)})
    print(str(metric))

loss_1 0.4000 (0.4500) loss_2 0.8000 (0.8000)
loss_1 0.6000 (0.5000) loss_2 0.7000 (0.7500)


In [27]:
# From: ./utils/tensor_board_logger.py
import os

from torch.utils.tensorboard.writer import SummaryWriter


def harmonic_mean(a, b):
    return 2 * (a * b) / (a + b)


class CSVLogger:
    """
    A simple logger that writes training metrics to a CSV file.

    Attributes:
        filename (str): The path to the output CSV file.
        file (file object): Open file handle for writing.
    """

    def __init__(self, filename):
        """
        Initializes the CSV logger and creates the output directory and file.

        Args:
            filename (str): Path to the CSV file for logging.
        """
        self.filename = filename
        os.makedirs(
            os.path.dirname(filename), exist_ok=True
        )  # Create folder if it doesn't exist
        self.file = open(filename, "w")
        self.file.write("epoch,base_acc,novel_acc,harmonic_mean\n")

    def log(self, epoch, base_acc, novel_acc):
        """
        Logs a training epoch's results including harmonic mean to the CSV file.

        Args:
            epoch (int): The epoch number.
            base_acc (float): Accuracy on base classes.
            novel_acc (float): Accuracy on novel classes.
        """
        hm = harmonic_mean(base_acc, novel_acc)
        self.file.write(f"{epoch},{base_acc},{novel_acc},{hm}\n")
        self.file.flush()
        print(
            f"Logged: epoch {epoch}, base_acc {base_acc}, novel_acc {novel_acc}, harmonic_mean {hm}"
        )

    def close(self):
        """
        Closes the CSV file.
        """
        self.file.close()


class BaseAndNovelMetrics:
    """
    Tracks base and novel accuracy values and their harmonic mean across epochs.

    Attributes:
        tmp (List[Tuple[int, float, float, float]]): Logged metrics per epoch.
    """

    def __init__(self):
        self.tmp = []

    def update(self, epoch, base_acc, novel_acc):
        """
        Records a new set of metrics for a specific epoch.

        Args:
            epoch (int): Epoch number.
            base_acc (float): Accuracy on base classes.
            novel_acc (float): Accuracy on novel classes.
        """
        self.tmp.append(
            (epoch, base_acc, novel_acc, harmonic_mean(base_acc, novel_acc))
        )

    def get_metrics(self):
        """
        Retrieves the collected metrics.

        Returns:
            List[Tuple[int, float, float, float]] or None: Logged metrics or None if empty.
        """
        if len(self.tmp) == 0:
            return None
        return self.tmp


class TensorboardLogger:
    """
    Handles logging of training and evaluation metrics to TensorBoard and CSV.

    Attributes:
        writer (SummaryWriter): TensorBoard writer instance.
        csv_logger (CSVLogger): CSV logger instance for persistent metric storage.
        hparams (dict): Hyperparameters dictionary.
        base_and_novel_metrics (BaseAndNovelMetrics): Metric tracker.
    """

    def __init__(self, writer: SummaryWriter):
        """
        Initializes the TensorboardLogger.

        Args:
            writer (SummaryWriter): TensorBoard writer instance.
        """
        self.writer = writer
        self.csv_logger = CSVLogger(f"{writer.log_dir}/metrics.csv")
        self.hparams = None
        self.base_and_novel_metrics = BaseAndNovelMetrics()

    def log_hparams(self, hparams: dict):
        """
        Stores hyperparameters for later logging.

        Args:
            hparams (dict): Hyperparameters dictionary.
        """
        self.hparams = hparams

    def log_training_base(self, epoch, lr, ce_loss, acc, kl_loss, total_loss):
        """
        Logs training metrics for the base training phase.

        Args:
            epoch (int): Current epoch.
            lr (float): Learning rate.
            ce_loss (float): Cross entropy loss.
            acc (float): Accuracy.
            kl_loss (float or None): KL divergence loss, optional.
            total_loss (float): Total loss.
        """
        self.writer.add_scalar("learning_rate", lr, epoch)
        self.writer.add_scalar("train_base/ce_loss", ce_loss, epoch)
        self.writer.add_scalar("train_base/ce_accuracy", acc, epoch)
        if kl_loss is not None:
            self.writer.add_scalar("train_base/kl_loss", kl_loss, epoch)
        self.writer.add_scalar("train_base/total_loss", total_loss, epoch)

    def log_training_adv(
        self, epoch, lambda_adv, ce_loss, acc, adv_loss, total_loss, kl_loss=None
    ):
        """
        Logs training metrics for the adversarial training phase.

        Args:
            epoch (int): Current epoch.
            lambda_adv (float): Adversarial loss weight.
            ce_loss (float): Cross entropy loss.
            acc (float): Accuracy.
            adv_loss (float): Adversarial loss.
            total_loss (float): Total loss.
            kl_loss (float or None): KL divergence loss, optional.
        """
        self.writer.add_scalar("lambda_adv", lambda_adv, epoch)
        self.writer.add_scalar("train_adv/ce_loss", ce_loss, epoch)
        self.writer.add_scalar("train_adv/ce_accuracy", acc, epoch)
        self.writer.add_scalar("train_adv/mlp_loss", adv_loss, epoch)
        if kl_loss is not None:
            self.writer.add_scalar("train_adv/kl_loss", kl_loss, epoch)
        self.writer.add_scalar("train_adv/total_loss", total_loss, epoch)

    def log_validation(
        self, epoch, base_loss, base_acc, novel_loss, novel_acc, is_adv=False
    ):
        """
        Logs validation metrics.

        Args:
            epoch (int): Current epoch.
            base_loss (float): Loss on base classes.
            base_acc (float): Accuracy on base classes.
            novel_loss (float): Loss on novel classes.
            novel_acc (float): Accuracy on novel classes.
            is_adv (bool): Whether validation is adversarial.
        """
        self.writer.add_scalar(f"validation_base/loss", base_loss, epoch)
        self.writer.add_scalar(f"validation_base/accuracy", base_acc, epoch)
        self.writer.add_scalar(f"validation_novel/loss", novel_loss, epoch)
        self.writer.add_scalar(f"validation_novel/accuracy", novel_acc, epoch)

        prefix = "validation_adv" if is_adv else "validation_ce"
        self.writer.add_scalar(f"{prefix}_base/loss", base_loss, epoch)
        self.writer.add_scalar(f"{prefix}_base/accuracy", base_acc, epoch)
        self.writer.add_scalar(f"{prefix}_novel/loss", novel_loss, epoch)
        self.writer.add_scalar(f"{prefix}_novel/accuracy", novel_acc, epoch)

    def log_final_metrics(self, tag, base_acc, novel_acc, step):
        """
        Logs final accuracy metrics and updates CSV and metric tracker.

        Args:
            tag (str): Tag name for TensorBoard.
            base_acc (float): Accuracy on base classes.
            novel_acc (float): Accuracy on novel classes.
            step (int): Training step or epoch index.
        """
        harmonic = harmonic_mean(base_acc, novel_acc)
        self.writer.add_scalars(
            tag,
            {
                "Harmonic Mean": harmonic,
                "Base Accuracy": base_acc,
                "Novel Accuracy": novel_acc,
            },
            global_step=step + 1,
        )

        self.csv_logger.log(step + 1, base_acc, novel_acc)

        if self.hparams is not None:
            self.base_and_novel_metrics.update(step + 1, base_acc, novel_acc)

        self.writer.flush()

    def log_test_accuracy(self, step, acc, label):
        """
        Logs test accuracy for a given label.

        Args:
            step (int): Step or epoch index.
            acc (float): Accuracy value.
            label (str): Label name for the accuracy metric.
        """
        self.writer.add_scalar(f"{label}/accuracy", acc, step)

    def close(self):
        """
        Closes the logger, writes hyperparameters and final metrics if available.
        """
        metrics = self.base_and_novel_metrics.get_metrics() or []

        """
        metric_dict = {
            "base_acc_after_base": metrics[0][1] if metrics else 0,
            "novel_acc_after_base": metrics[0][2] if metrics else 0,
            "harmonic_mean_after_base": metrics[0][3] if metrics else 0,
            "base_acc_after_adv": metrics[1][1] if metrics else 0,
            "novel_acc_after_adv": metrics[1][2] if metrics else 0,
            "harmonic_mean_after_adv": metrics[1][3] if metrics else 0,
        }"""

        if self.hparams is not None and metrics:

            tmp = {}
            # Set the prefix based on whether metrics are from base phase (index 0) or adversarial phase (index 1)
            for idx, m in enumerate(metrics):
                prefix = "after_base" if idx == 0 else "after_adv"
                tmp[f"epoch_{prefix}"] = m[0]
                tmp[f"base_acc_{prefix}"] = m[1]
                tmp[f"novel_acc_{prefix}"] = m[2]
                tmp[f"harmonic_mean_{prefix}"] = m[3]

            self.writer.add_hparams(
                hparam_dict=self.hparams,
                metric_dict=tmp,
            )
        self.writer.close()
        self.csv_logger.close()

In [28]:
# From: ./utils/training_coop.py
"""
This module provides training, evaluation, and testing functions for the CoOp model and standard CLIP evaluation,
including fine-tuning and zero-shot classification on image datasets.
"""
from clip.model import CLIP
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import clip

# from model.coop.custom_clip import CustomCLIPCoOp
# from utils.datasets import ContiguousLabelDataset, CLASS_NAMES


@torch.no_grad()
def eval_step(model, dataset, cost_function, new_classnames, batch_size=32, device="cuda"):
    """
    Evaluates the model on a given dataset using cross-entropy loss.

    Args:
        model (nn.Module): The model to evaluate.
        dataset (Dataset): Dataset to evaluate on.
        cost_function (Callable): Loss function to use.
        batch_size (int): Batch size for evaluation.
        device (str): Computation device ("cuda" or "cpu").
        new_classnames (List[int] or None): Optional list of class indices to temporarily substitute for evaluation.

    Returns:
        Tuple[float, float]: Average loss and accuracy.
    """
    model.eval()

    tmp_dataset = ContiguousLabelDataset(dataset, new_classnames)
    dataloader = DataLoader(tmp_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    total_loss = 0.0
    correct = 0
    total = 0
    if new_classnames is not None:
        new_classnames = [CLASS_NAMES[c] for c in new_classnames]
        with model.temporary_classnames(new_classnames):
            correct, total, total_loss = walk_the_dataset(correct, cost_function, dataloader, device, model, total,
                                                          total_loss)

    else:
        correct, total, total_loss = walk_the_dataset(correct, cost_function, dataloader, device, model, total,
                                                      total_loss)
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


def walk_the_dataset(correct, cost_function, dataloader, device, model, total, total_loss):
    """
    Iterates over the dataset and computes cumulative loss and accuracy.

    Args:
        correct (int): Running count of correct predictions.
        cost_function (Callable): Loss function used.
        dataloader (DataLoader): DataLoader for the dataset.
        device (str): Computation device.
        model (nn.Module): Model being evaluated.
        total (int): Total number of samples evaluated so far.
        total_loss (float): Accumulated loss value.

    Returns:
        Tuple[int, int, float]: Updated correct, total, and total_loss.
    """
    for images, targets in tqdm(dataloader, desc="Validation", position=1, leave=False):
        images = images.to(device)
        targets = targets.to(device)

        loss, logits = model(images, targets)

        total_loss += loss.item() * targets.size(0)
        predictions = logits.argmax(dim=-1)
        correct += (predictions == targets).sum().item()
        total += targets.size(0)
    return correct, total, total_loss


def training_step(model: CustomCLIPCoOp, dataset, optimizer, batch_size, classnames, device="cuda"):
    """
    Performs one full training epoch for the CoOp model.

    Args:
        model (CustomCLIPCoOp): The model to train.
        dataset (Dataset): Dataset to train on.
        optimizer (Optimizer): Optimizer used for updating model parameters.
        batch_size (int): Batch size for training.
        device (str): Computation device.

    Returns:
        Tuple[float, float]: Average training loss and accuracy.
    """
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # Set the network to training mode

    model.train()

    tmp_dataset = ContiguousLabelDataset(dataset, classnames)
    dataloader = DataLoader(tmp_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    pbar = tqdm(dataloader, desc="Training", position=1, leave=False)
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        # Load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)
        # debug if inputs and targets are taken correctly by the dataloader
        #print(inputs.shape)
        #print(targets.shape)
        # Forward pass + loss computation
        loss, logits = model(inputs, targets)

        if torch.isnan(loss):
            print("⚠️ NaN loss encountered!")
            #print("Logits:", logits)
            print("Targets:", targets)

        # Backward pass
        loss.backward()

        # Parameters update
        optimizer.step()

        # Gradients reset
        optimizer.zero_grad()

        # Fetch prediction and loss value
        samples += inputs.shape[0]
        cumulative_loss += loss.item() * inputs.shape[0]
        _, predicted = logits.max(dim=1)  # max() returns (maximum_value, index_of_maximum_value)

        # Compute training accuracy
        cumulative_accuracy += predicted.eq(targets).sum().item()

        pbar.set_postfix(train_loss=loss.item(), train_acc=cumulative_accuracy / samples )
        pbar.update(1)

    return cumulative_loss / samples, cumulative_accuracy / samples


@torch.no_grad()
def test_step(model, dataset, batch_size, device, categories, label="test", base=False):
    """
    Evaluates the model using either fine-tuned or base (zero-shot) strategy.

    Args:
        model (nn.Module): The model to test.
        dataset (Dataset): Dataset for testing.
        batch_size (int): Batch size for testing.
        device (str): Device used for computation.
        label (str): Label for the progress bar.
        base (bool): Whether to use zero-shot CLIP instead of the fine-tuned model.

    Returns:
        float: Accuracy score.
    """
    if not base:
        return finetuned_test_step(model, dataset, batch_size, device, categories, label)
    else:
        return base_test_step(model, dataset, categories, batch_size, device, label)


@torch.no_grad()
def finetuned_test_step(model: CustomCLIPCoOp, dataset, batch_size, device, categories, label="test"):
    """
    Evaluates a fine-tuned CustomCLIPCoOp model on the given dataset.

    Args:
        model (CustomCLIPCoOp): Fine-tuned CoOp model.
        dataset (Dataset): Dataset for evaluation.
        batch_size (int): Batch size.
        device (str): Computation device.
        label (str): Label for progress display.

    Returns:
        float: Accuracy of the model on the dataset.
    """
    model.eval()

    tmp_dataset = ContiguousLabelDataset(dataset, categories)
    dataloader = DataLoader(tmp_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    correct = 0
    total = 0

    for images, targets in tqdm(dataloader, desc=label):
        images = images.to(device)
        targets = targets.to(device)

        logits = model(images)
        predictions = logits.argmax(dim=-1)

        correct += (predictions == targets).sum().item()
        total += targets.size(0)

    accuracy = correct / total
    return accuracy


@torch.no_grad()  # we don't want gradients
def base_test_step(model: CLIP, dataset, categories, batch_size, device, label=""):
    """
    Evaluates a zero-shot CLIP model using cosine similarity between image and text embeddings.

    Args:
        model (CLIP): Pretrained CLIP model.
        dataset (Dataset): Dataset to evaluate.
        categories (List[int]): List of category indices to evaluate.
        batch_size (int): Batch size for evaluation.
        device (str): Computation device.
        label (str): Optional label for progress bar.

    Returns:
        float: Accuracy of zero-shot CLIP classification.
    """
    # let's set the model in evaluation mode
    model.eval()

    # Remap labels into a contiguous set starting from zero
    contig_cat2idx = {cat: idx for idx, cat in enumerate(categories)}

    # here we apply the standard CLIP template used for oxford flowers to all categories
    # and immediately tokenize each sentence (convert natural language into numbers - feel free to print the text input to inspect them)
    text_inputs = clip.tokenize(
        [f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in categories]
    ).to(device)

    # we can encode the text features once as they are shared for all images
    # therefore we do it outside the evaluation loop
    text_features = model.encode_text(text_inputs)
    # and here we normalize them (standard pratice with CLIP)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # simple dataloader creation
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # here we store the number of correct predictions we will make
    correct_predictions = 0
    for image, target in tqdm(dataloader, desc=label):
        # base categories range from 0 to 50, while novel ones from 51 to 101
        # therefore we must map categories to the [0, 50], otherwise we will have wrong predictions
        # Map targets in contiguous set starting from zero
        # Labels needs to be .long() in pytorch
        target = torch.Tensor([contig_cat2idx[t.item()] for t in target]).long()

        image = image.to(device)
        target = target.to(device)

        # forward image through CLIP image encoder
        image_features = model.encode_image(image)
        # and normalize
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # here cosine similarity between image and text features and keep the argmax for every row (every image)
        predicted_class = (image_features @ text_features.T).argmax(dim=-1)
        # now we check which are correct, and sum them (False == 0, True == 1)
        correct_predictions += (predicted_class == target).sum().item()

    # and now we compute the accuracy
    accuracy = correct_predictions / len(dataset)
    return accuracy

In [ ]:
# From: ./main.py
import yaml
import argparse

# from training_systems.cocoop import CoCoOpSystem
# from training_systems.coop import CoOpSystem
import torch
import os
from datetime import datetime
import pickle
from collections import Counter


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--device', default=os.getenv("DEVICE", "cuda:0"))
    parser.add_argument('--run_name', default=f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
    parser.add_argument('--using_coop', default=False, type=lambda x: x.lower() in ("1", "true", "yes", "true"))
    parser.add_argument('--config', default="train_config.yaml")
    parser.add_argument('--debug', default=True, type=lambda x: x.lower() in ("1", "true", "yes", "true"))
    return parser.parse_args()


if __name__ == "__main__":
    # Assign parsed arguments to variables
    # Display which device is being used
    # Handle MPS backend by setting default tensor type to float32
    # Indicate whether CoOp or CoCoOp is used for training
    # Load training configuration from YAML file

    runs_yaml = ["test_kl+adv"]
    runs_config = [{
        "coop": {
            "batch_size": 10,
            "learning_rate": 0.002,
            "weight_decay": 0.0001,
            "momentum": 0.9,
            "epochs": 20,
            "n_ctx": 8,
            "ctx_init": "",
            "class_token_position": "end",
            "csc": False
        },
        "cocoop": {
            "report": True,
            "pat": False,
            "cnn_model": "ViT-B/16",
            "test_batch_size": 10,
            "train_base_checkpoint_path": None, # Cant have outside files in the jupyter
            "optimizer_configs": [
                {
                    "prompt_lr": 0.002,
                    "weight_decay": 0.0001,
                    "momentum": 0.9
                },
                {
                    "prompt_lr": 0.002,
                    "mlp_lr": 0.005,
                    "weight_decay": 0.0005,
                    "momentum": 0.8
                }
            ],
            "skip_tests": [True, True, False],
            "prompt_learner_opt": {
                "n_ctx": 4,
                "ctx_init": "",
                "class_token_position": "end",
                "csc": False
            },
            "kl_loss_opt": {
                "lambda_kl": [0.1, 0.1],
                "using_kl": [True, False]
            },
            "adv_training_opt": {
                "adv_training_epochs": 40,
                "batch_size": 10,
                "warmup_lambda_adv": 10,
                "lambda_adv": 3,
                "grl_lambda": 1,
                "mlp_opt": {
                    "hidden_structure": [563, 256, 128]
                },
                "prompt_learner_warmup_epochs": 3,
                "ignore_no_improvement": True,
                "use_bias_ctx": True
            },
            "base_training_opt": {
                "epochs": 5,
                "batch_size": 10,
                "warmup_epoch": 0,
                "warmup_cons_lr": 1e-5
            },
            "clustering_opt": {
                "n_clusters": 4,
                "variance": 0.95,
                "vision_encoder": "ViT-B/32",
                "clustering_type": "semantic"
            }
            }
        },]

    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda:0" if torch.cuda.is_available() else "cpu")
    debug = False
    use_coop = False

    for run_id, run_yaml in enumerate(runs_yaml):
        run_name = "jupyter_notebook_"+ run_yaml+ "_" + datetime.now().strftime("%Y%m%d_%H%M%S")

        print(f"Using device: {device}")

        if torch.backends.mps.is_available():
            print("\u26a0\ufe0f Forcing float32 due to MPS limitations")
            torch.set_default_dtype(torch.float32)

        print(f"Using {'CoOp' if use_coop else 'CoCoOp'} for training")

        config = runs_config[run_id] if isinstance(runs_config, list) else runs_config

        if use_coop:
            coop_cfg = config['coop']
            train_sys = CoOpSystem(
                device=device,
                run_name=run_name,
                **coop_cfg
            )
        else:
            cocoop_cfg = config['cocoop']
            train_sys = CoCoOpSystem(
                device=device,
                run_name=run_name,
                debug=debug,
                hparams_file=run_yaml,
                **cocoop_cfg
            )
        print(f"\nTraining system initialized with run name: {train_sys.run_name}")
        train_sys.train()

Using device: cuda:0
Using CoCoOp for training
run_name: jupyter_notebook_test_kl+adv_20250723_130854, using ViT-B/16, pat: False
BATCH SIZES:  10 10 10
patience disabled
Base classes length: 51, Novel classes length: 51
Train base length: 510, Val base length: 510, Test base length: 2473
Val novel length: 510, Test novel length: 3676
mlp adversary struct:  AdversarialMLP(
  (model): Sequential(
    (0): Linear(in_features=1024, out_features=563, bias=True)
    (1): ReLU()
    (2): ResidualBlock(
      (linear): Linear(in_features=563, out_features=256, bias=True)
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
      (drop): Dropout(p=0.2, inplace=False)
      (residual): Linear(in_features=563, out_features=256, bias=True)
    )
    (3): ResidualBlock(
      (linear): Linear(in_features=256, out_features=128, bias=True)
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU()
      (drop): Dropout(p=0.2, inplace=

Processing classes: 100%|██████████| 51/51 [01:55<00:00,  2.27s/it]


Reduced feature shape: (51, 31), Variance explained: 0.9532072318917544
Cluster 0 count: 19
Cluster 1 count: 15
Cluster 2 count: 8
Cluster 3 count: 9

Training system initialized with run name: jupyter_notebook_test_kl+adv_20250723_130854


Base Training:   0%|          | 0/5 [00:00<?, ?it/s]

meta_net_dtype: torch.float16, im_features dtype: torch.float16
meta_net_dtype: torch.float16, im_features dtype: torch.float16


...

meta_net_dtype: torch.float16, im_features dtype: torch.float16


meta_net_dtype: torch.float16, im_features dtype: torch.float16


meta_net_dtype: torch.float16, im_features dtype: torch.float16


meta_net_dtype: torch.float16, im_features dtype: torch.float16


meta_net_dtype: torch.float16, im_features dtype: torch.float16
